# Part 4

In this part, I will build my models, using 4 types of Machine Learning algortihms (SVR, Random Forest,Gradient Boost, AdaBoost).

## Building the Models

In [ ]:
import pandas as pd
import numpy as np
#Connect to API
from datetime import date
import datetime
from pandas.testing import assert_frame_equal
#Library for Plotting
import seaborn as sns
sns.set(style="darkgrid")
import matplotlib.pyplot as plt
%matplotlib inline

#Library for Data Preprocessing and Cleaning
from sklearn.exceptions import ConvergenceWarning
from sklearn.model_selection import train_test_split
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer

import time
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import RandomizedSearchCV,TimeSeriesSplit, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

C:\Users\Joyce\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 1.Baselines

In [ ]:
train_df= pd.read_csv('train_lag_dummies_59.csv', index_col='quarter')
train_df.drop('Unnamed: 0', axis=1, inplace=True)
test_df= pd.read_csv('test_lag_dummies_59.csv',index_col='quarter')
test_df.drop('Unnamed: 0', axis=1, inplace=True)
train_df.head()

,Categ_name,GT,GT_lag1,GT_lag2,DI,DI_lag1,DI_lag2,3m Bond,3m Bond_lag1,3m Bond_lag2,...,Restaurants: 276,Small Kitchen Appliances: 1292,Sporting Goods: 263,Tobacco Products: 123,Transportation & Logistics: 50,Vehicle Fuels & Lubricants: 1269,Vehicle Maintenance: 138,Vehicle Parts & Accessories: 89,Water Supply & Treatment: 1349,Wine: 405
quarter,,,,,,,,,,,,,,,,,,,,,
2005Q1,Air Travel: 203,2.525412,0.000000,0.000000,0.286653,0.000000,0.000000,0.332819,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
2005Q2,Air Travel: 203,-0.367123,2.525412,0.000000,1.226200,0.286653,0.000000,0.199119,0.332819,0.000000,...,0,0,0,0,0,0,0,0,0,0
2005Q3,Air Travel: 203,0.818342,-0.367123,2.525412,0.756426,1.226200,0.286653,-0.373748,0.199119,0.332819,...,0,0,0,0,0,0,0,0,0,0
2005Q4,Air Travel: 203,-2.927727,0.818342,-0.367123,1.312595,0.756426,1.226200,0.032748,-0.373748,0.199119,...,0,0,0,0,0,0,0,0,0,0
2006Q1,Air Travel: 203,1.908970,-2.927727,0.818342,0.999412,1.312595,0.756426,0.199491,0.032748,-0.373748,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# replace NaN values with zero
train_df = train_df.fillna(0)
train_df.isna().sum()

Categ_name                          0
GT                                  0
GT_lag1                             0
GT_lag2                             0
DI                                  0
                                   ..
Vehicle Fuels & Lubricants: 1269    0
Vehicle Maintenance: 138            0
Vehicle Parts & Accessories: 89     0
Water Supply & Treatment: 1349      0
Wine: 405                           0
Length: 65, dtype: int64

In [ ]:
test_df = test_df.fillna(0)
test_df.isna().sum()

Categ_name                          0
GT                                  0
GT_lag1                             0
GT_lag2                             0
DI                                  0
                                   ..
Vehicle Fuels & Lubricants: 1269    0
Vehicle Maintenance: 138            0
Vehicle Parts & Accessories: 89     0
Water Supply & Treatment: 1349      0
Wine: 405                           0
Length: 65, dtype: int64

## 1.1 **Persistence baseline**

We will use past values as forecasts of the former. The `shift()` method creates a new series with values shifted one quarter ahead:
We will do that for each categorie.

In [ ]:
test_df_base = test_df.copy()
test_df_base.head()

,Categ_name,GT,GT_lag1,GT_lag2,DI,DI_lag1,DI_lag2,3m Bond,3m Bond_lag1,3m Bond_lag2,...,Restaurants: 276,Small Kitchen Appliances: 1292,Sporting Goods: 263,Tobacco Products: 123,Transportation & Logistics: 50,Vehicle Fuels & Lubricants: 1269,Vehicle Maintenance: 138,Vehicle Parts & Accessories: 89,Water Supply & Treatment: 1349,Wine: 405
quarter,,,,,,,,,,,,,,,,,,,,,
2017Q1,Air Travel: 203,0.628668,0.000000,0.000000,1.911960,0.000000,0.000000,0.202656,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
2017Q2,Air Travel: 203,0.344156,0.628668,0.000000,1.593379,1.911960,0.000000,0.129086,0.202656,0.000000,...,0,0,0,0,0,0,0,0,0,0
2017Q3,Air Travel: 203,0.581249,0.344156,0.628668,1.847164,1.593379,1.911960,0.235197,0.129086,0.202656,...,0,0,0,0,0,0,0,0,0,0
2017Q4,Air Travel: 203,-1.125820,0.581249,0.344156,1.998355,1.847164,1.593379,0.559189,0.235197,0.129086,...,0,0,0,0,0,0,0,0,0,0
2018Q1,Air Travel: 203,0.723505,-1.125820,0.581249,2.041553,1.998355,1.847164,0.210438,0.559189,0.235197,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
from sklearn.metrics import mean_squared_error

We would like to calculate the persistence baseline for each category.

In [ ]:
rmse_persi_df=[]
def rmse_persi( g ):
    rmse_persistence = np.sqrt(mean_squared_error(g['CS'][1:], g['CS'].shift()[1:]))
    rmse_persi_df.append(rmse_persistence)
    return pd.Series(dict( rmse = rmse_persistence) )

In [ ]:
test_df_base.groupby( 'Categ_name' ).apply(rmse_persi).reset_index()

,Categ_name,rmse
0,Air Travel: 203,319.137205
1,Animal Products & Services: 882,53.086931
2,Apartments & Residential Rentals: 378,460.738176
3,Apparel: 68,214.250113
4,Audio Equipment: 361,40.921334
5,Auto Insurance: 467,18.782379
6,Automotive Industry: 1190,524.779319
7,Baked Goods: 907,42.589774
8,Beer: 404,45.858478
9,Bicycles & Accessories: 1191,11.372481


In [ ]:
rmse_persi_all = np.sqrt(mean_squared_error(test_df_base['CS'][1:], test_df_base['CS'].shift()[1:]))
rmse_persi_all

4936.789529657899

In [ ]:
def get_mda(y, yhat):
    """Mean Directional Accuracy, as per:
    https://www.wikiwand.com/en/Mean_Directional_Accuracy
    """
    a = np.sign(np.diff(y))
    b = np.sign(np.diff(yhat))
    return np.sum(a == b)/a.shape[0]

mda_persi_df=[]
def mda_persi( g ):
    mda_persistence = get_mda(g['CS'][1:], g['CS'].shift()[1:])
    mda_persi_df.append(mda_persistence)
    return pd.Series(dict( mda = mda_persistence) )

In [ ]:
test_df_base.groupby( 'Categ_name' ).apply(mda_persi).reset_index()

,Categ_name,mda
0,Air Travel: 203,0.375
1,Animal Products & Services: 882,0.625
2,Apartments & Residential Rentals: 378,0.875
3,Apparel: 68,0.750
4,Audio Equipment: 361,0.375
5,Auto Insurance: 467,0.875
6,Automotive Industry: 1190,0.625
7,Baked Goods: 907,0.750
8,Beer: 404,0.625
9,Bicycles & Accessories: 1191,0.375


In [ ]:
mda_persi_all=get_mda(test_df_base['CS'][1:], test_df_base['CS'].shift()[1:])
mda_persi_all

0.6144859813084113

In [ ]:
persi_base_df=pd.DataFrame({'rmse':rmse_persi_df, 'mda':mda_persi_df})
persi_base_df

,rmse,mda
0,319.137205,0.375
1,53.086931,0.625
2,460.738176,0.875
3,214.250113,0.750
4,40.921334,0.375
5,18.782379,0.875
6,524.779319,0.625
7,42.589774,0.750
8,45.858478,0.625
9,11.372481,0.375


## 1.2 **Mean baseline**

The mean baseline calculates the mean of the series in the training data and predicts the value for each test observation. I must calculate the mean for each category. Then, I will predict the mean for the test set for each category and compare it versus the baseline.

In [ ]:
train_mean = train_df['CS'].mean()
train_mean

4199.623356926188

In [ ]:
rmse_mean_df=[]
def rmse_mean( j ):
    train_mean = train_df['CS'].mean()
    rmse_mean = np.sqrt(mean_squared_error(j['CS'], [train_mean]*j.shape[0]))
    rmse_mean_df.append(rmse_mean)
    return pd.Series(dict( rmse_mean = rmse_mean) )

In [ ]:
test_df_base.groupby( 'Categ_name' ).apply(rmse_mean).reset_index()

,Categ_name,rmse_mean
0,Air Travel: 203,2078.010352
1,Animal Products & Services: 882,2542.803229
2,Apartments & Residential Rentals: 378,70301.117373
3,Apparel: 68,12792.986195
4,Audio Equipment: 361,2944.584473
5,Auto Insurance: 467,3378.308310
6,Automotive Industry: 1190,7931.388195
7,Baked Goods: 907,87.947711
8,Beer: 404,2844.564496
9,Bicycles & Accessories: 1191,3805.439697


In [ ]:
mda_mean_df=[]
def mda_mean( g ):
    train_mean = train_df['CS'].mean()
    mda_mean = get_mda(g['CS'], [train_mean]*g.shape[0])
    mda_mean_df.append(mda_mean)
    return pd.Series(dict( mda_mean = mda_mean) )

In [ ]:
test_df_base.groupby('Categ_name').apply(mda_mean).reset_index()

,Categ_name,mda_mean
0,Air Travel: 203,0.000000
1,Animal Products & Services: 882,0.000000
2,Apartments & Residential Rentals: 378,0.000000
3,Apparel: 68,0.000000
4,Audio Equipment: 361,0.000000
5,Auto Insurance: 467,0.000000
6,Automotive Industry: 1190,0.000000
7,Baked Goods: 907,0.000000
8,Beer: 404,0.000000
9,Bicycles & Accessories: 1191,0.000000


In [ ]:
mean_base_df=pd.DataFrame({'rmse':rmse_mean_df, 'mda':mda_mean_df})
mean_base_df

,rmse,mda
0,2078.010352,0.000000
1,2542.803229,0.000000
2,70301.117373,0.000000
3,12792.986195,0.000000
4,2944.584473,0.000000
5,3378.308310,0.000000
6,7931.388195,0.000000
7,87.947711,0.000000
8,2844.564496,0.000000
9,3805.439697,0.000000


# 2. Training Machine Learning  models

## 2.1 Build models with nowcasting and lagged values

In [ ]:
# Drop target variable which is CS
Xtrain=train_df.drop(['CS', 'Categ_name'], axis=1)
ytrain=train_df[['CS']].copy()

# Copy without the GT data
Xtrain_light=train_df.copy().drop(['CS', 'Categ_name','GT','GT_lag1','GT_lag2'], axis=1)
ytrain_light=train_df[['CS']].copy()

In [ ]:
Xtrain_light.head()

,DI,DI_lag1,DI_lag2,3m Bond,3m Bond_lag1,3m Bond_lag2,cci,cci_lag1,cci_lag2,CPI,...,Restaurants: 276,Small Kitchen Appliances: 1292,Sporting Goods: 263,Tobacco Products: 123,Transportation & Logistics: 50,Vehicle Fuels & Lubricants: 1269,Vehicle Maintenance: 138,Vehicle Parts & Accessories: 89,Water Supply & Treatment: 1349,Wine: 405
quarter,,,,,,,,,,,,,,,,,,,,,
2005Q1,0.286653,0.000000,0.000000,0.332819,0.000000,0.000000,0.954549,0.000000,0.000000,-0.620977,...,0,0,0,0,0,0,0,0,0,0
2005Q2,1.226200,0.286653,0.000000,0.199119,0.332819,0.000000,0.822608,0.954549,0.000000,0.391964,...,0,0,0,0,0,0,0,0,0,0
2005Q3,0.756426,1.226200,0.286653,-0.373748,0.199119,0.332819,0.818727,0.822608,0.954549,-0.215800,...,0,0,0,0,0,0,0,0,0,0
2005Q4,1.312595,0.756426,1.226200,0.032748,-0.373748,0.199119,0.562606,0.818727,0.822608,-0.215800,...,0,0,0,0,0,0,0,0,0,0
2006Q1,0.999412,1.312595,0.756426,0.199491,0.032748,-0.373748,0.647980,0.562606,0.818727,-1.228741,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
Xtrain_light.columns.to_list()

['DI',
 'DI_lag1',
 'DI_lag2',
 '3m Bond',
 '3m Bond_lag1',
 '3m Bond_lag2',
 'cci',
 'cci_lag1',
 'cci_lag2',
 'CPI',
 'CPI_lag1',
 'CPI_lag2',
 'vix',
 'vix_lag1',
 'vix_lag2',
 'CS_lag1',
 'CS_lag2',
 'Air Travel: 203',
 'Animal Products & Services: 882',
 'Apartments & Residential Rentals: 378',
 'Apparel: 68',
 'Audio Equipment: 361',
 'Auto Insurance: 467',
 'Automotive Industry: 1190',
 'Baked Goods: 907',
 'Beer: 404',
 'Bicycles & Accessories: 1191',
 'Camera & Photo Equipment: 573',
 'Candy & Sweets: 906',
 'Clothing Accessories: 124',
 'Domestic Services: 472',
 'Drugs & Medications: 646',
 'Education: 74',
 'Electricity: 658',
 'Fruits & Vegetables: 908',
 'Health Insurance: 249',
 'Hobbies & Leisure: 65',
 'Home Financing: 466',
 'Home Furnishings: 270',
 'Home Insurance: 465',
 'Hotels & Accommodations: 179',
 'Kitchen & Dining: 951',
 'Liquor: 406',
 'Lottery & Sweepstakes: 364',
 'Mail & Package Delivery: 1150',
 'Major Kitchen Appliances: 1293',
 'Meat & Seafood: 909',

In [ ]:
Xtrain.head()

,GT,GT_lag1,GT_lag2,DI,DI_lag1,DI_lag2,3m Bond,3m Bond_lag1,3m Bond_lag2,cci,...,Restaurants: 276,Small Kitchen Appliances: 1292,Sporting Goods: 263,Tobacco Products: 123,Transportation & Logistics: 50,Vehicle Fuels & Lubricants: 1269,Vehicle Maintenance: 138,Vehicle Parts & Accessories: 89,Water Supply & Treatment: 1349,Wine: 405
quarter,,,,,,,,,,,,,,,,,,,,,
2005Q1,2.525412,0.000000,0.000000,0.286653,0.000000,0.000000,0.332819,0.000000,0.000000,0.954549,...,0,0,0,0,0,0,0,0,0,0
2005Q2,-0.367123,2.525412,0.000000,1.226200,0.286653,0.000000,0.199119,0.332819,0.000000,0.822608,...,0,0,0,0,0,0,0,0,0,0
2005Q3,0.818342,-0.367123,2.525412,0.756426,1.226200,0.286653,-0.373748,0.199119,0.332819,0.818727,...,0,0,0,0,0,0,0,0,0,0
2005Q4,-2.927727,0.818342,-0.367123,1.312595,0.756426,1.226200,0.032748,-0.373748,0.199119,0.562606,...,0,0,0,0,0,0,0,0,0,0
2006Q1,1.908970,-2.927727,0.818342,0.999412,1.312595,0.756426,0.199491,0.032748,-0.373748,0.647980,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
ytrain.head()

,CS
quarter,
2005Q1,3344
2005Q2,3332
2005Q3,3442
2005Q4,3435
2006Q1,3423


We do the same for the test set.

In [ ]:
# # Drop target variable which is CS
Xtest=test_df.copy().drop(['CS', 'Categ_name'], axis=1)
ytest=test_df[['CS']].copy()

# Copy without the GT data
Xtest_light=test_df.copy().drop(['CS', 'Categ_name','GT','GT_lag1','GT_lag2'], axis=1)
ytest_light=test_df[['CS']].copy()

In [ ]:
#creation of a function to display the RMSE scores
def display_scores(scores):
    print("Scores:", scores)
    print("Accuracy:", scores.mean())
    print("Standard deviation:", scores.std())

### 2.1.1 Random Forests

#### 2.1.1.1 *With GT*

#### *Model 1*

In [ ]:
# specify the hyperparameters and their values
# n_estimators = number of decision trees used in the random forest
# max depth :  maximum number of levels in the trees
# min_samples_Split = Minimum number of samples required to split a node
rf= RandomForestRegressor(random_state=20)
param_grid = [{'n_estimators': [1,2, 3, 5, 10, 20, 30, 40,50,100], 
              'max_depth': [2, 4, 6, 8, 10, None]}]

tscv = TimeSeriesSplit(n_splits=5)

# start a new timer
rf_start=time.time()

# we'll use tscv with 5 folds 
# we also want to get the train score for later
grid_search = GridSearchCV(estimator=rf, cv=tscv,
                           param_grid=param_grid,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# fit the estimator on the training set
grid_search.fit(Xtrain, ytrain)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=20,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None

In [ ]:
# the best model's RMSE
rf_rmse_score=np.sqrt(-grid_search.best_score_)
print(f'The best Random Forest model has a RMSE of: {rf_rmse_score}')

The best Random Forest model has a RMSE of: 782.0748840960143


In [ ]:
# the best random forest model
m_rf1 = grid_search.best_estimator_
m_rf1

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
                      oob_score=False, random_state=20, verbose=0,
                      warm_start=False)

Let's have a look at the accuracy scores for every model evaluated during the grid search

In [ ]:
# training and validation RMSE

val_scores = grid_search.cv_results_["mean_test_score"]
train_scores = grid_search.cv_results_["mean_train_score"]
params = grid_search.cv_results_["params"]

for val_score, train_score, param in sorted(zip(val_scores, train_scores, params)):
    print(np.sqrt(-val_score), np.sqrt(-train_score), param)

2147.4963153985336 1841.47364728361 {'max_depth': 2, 'n_estimators': 2}
2140.7139707829024 1905.1764582774076 {'max_depth': 2, 'n_estimators': 3}
2139.8119495214073 1887.8511992515068 {'max_depth': 2, 'n_estimators': 1}
2131.86205091079 1795.7758080186045 {'max_depth': 2, 'n_estimators': 5}
2109.1700312069725 1791.160307713291 {'max_depth': 2, 'n_estimators': 10}
2098.196876792265 1757.9273694376757 {'max_depth': 2, 'n_estimators': 30}
2097.796790718645 1755.0879848110737 {'max_depth': 2, 'n_estimators': 40}
2095.730885988335 1750.102351449126 {'max_depth': 2, 'n_estimators': 100}
2094.3134616987163 1776.6839333114303 {'max_depth': 2, 'n_estimators': 20}
2093.9082726110073 1763.2445312239847 {'max_depth': 2, 'n_estimators': 50}
1128.8226480794478 579.0121378272569 {'max_depth': 6, 'n_estimators': 1}
1082.7779942314958 495.32939154318495 {'max_depth': 8, 'n_estimators': 1}
1070.371200194702 491.9416888102006 {'max_depth': 10, 'n_estimators': 1}
1061.6012648030824 735.6760673030767 {'max

The error rate for the validation set is higher than the training set.It overfits.

In [ ]:
# make predictions
rf1_yhat = m_rf1.predict(Xtrain)

# Calculate how much time it took to tune the hyperparameters and train the model
rf1_duration = time.time() - rf_start
print(f'The Random Forest model took {rf1_duration:.3f} seconds')

The Random Forest model took 94.343 seconds


We will save the model

In [ ]:
import os
from joblib import dump

# create a folder where all trained models will be kept

if not os.path.exists("Diss_models"):
    os.makedirs("Diss_models")
    
dump(m_rf1, 'Diss_models/rf1_59.joblib')

['Diss_models/rf1_59.joblib']

#### *Model 2*

In [ ]:
# specify the hyperparameters and their values
# n_estimators = number of decision trees used in the random forest
# max depth :  maximum number of levels in the trees
# min_samples_Split = Minimum number of samples required to split a node
rf2= RandomForestRegressor(random_state=20)
param_grid2 = [{'n_estimators': [1,2,3,4,5,7,10], 
              'max_depth': [2,5,8, 10, 15,20,25,30,40,50,None]}]

tscv = TimeSeriesSplit(n_splits=5)

# start a new timer
rf_start2=time.time()

# we'll use tscv with 5 folds 
# we also want to get the train score for later
grid_search2 = GridSearchCV(estimator=rf2, cv=tscv,
                           param_grid=param_grid2,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# fit the estimator on the training set
grid_search2.fit(Xtrain, ytrain)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=20,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None

In [ ]:
# the best model's RMSE
rf_rmse_score2=np.sqrt(-grid_search2.best_score_)
print(f'The best Random Forest model has a RMSE of: {rf_rmse_score2}')

The best Random Forest model has a RMSE of: 782.0748840960143


The RMSE of this model is better than the last one. 

In [ ]:
# the best random forest model
m_rf2 = grid_search2.best_estimator_
m_rf2

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
                      oob_score=False, random_state=20, verbose=0,
                      warm_start=False)

Best model has a max depth of 8and min samples split of 2.

In [ ]:
# training and validation RMSE

val_scores2 = grid_search2.cv_results_["mean_test_score"]
train_scores2 = grid_search2.cv_results_["mean_train_score"]
params2 = grid_search2.cv_results_["params"]

for val_score, train_score, param in zip(val_scores2, train_scores2, params2):
    print(np.sqrt(-val_score), np.sqrt(-train_score), param)

2139.8119495214073 1887.8511992515068 {'max_depth': 2, 'n_estimators': 1}
2147.4963153985336 1841.47364728361 {'max_depth': 2, 'n_estimators': 2}
2140.7139707829024 1905.1764582774076 {'max_depth': 2, 'n_estimators': 3}
2139.0859465412664 1832.9565281202622 {'max_depth': 2, 'n_estimators': 4}
2131.86205091079 1795.7758080186045 {'max_depth': 2, 'n_estimators': 5}
2120.2457912768864 1768.9998725605196 {'max_depth': 2, 'n_estimators': 7}
2109.1700312069725 1791.160307713291 {'max_depth': 2, 'n_estimators': 10}
955.9449332936954 641.0123317524003 {'max_depth': 5, 'n_estimators': 1}
931.5278420973412 782.55273358861 {'max_depth': 5, 'n_estimators': 2}
871.6597921314614 889.3665731477178 {'max_depth': 5, 'n_estimators': 3}
831.2669882368945 766.7246000498709 {'max_depth': 5, 'n_estimators': 4}
820.433188283802 668.6269620327848 {'max_depth': 5, 'n_estimators': 5}
808.2280435621592 590.8872413083095 {'max_depth': 5, 'n_estimators': 7}
854.4501969364122 648.1534861060592 {'max_depth': 5, 'n_e

In [ ]:
# training and validation RMSE
m_rf2_val_df=pd.DataFrame({'val': np.sqrt(-val_scores2),'train':  np.sqrt(-train_scores2), 'param': params2} )
m_rf2_val_df.sort_values('val', inplace=True)
m_rf2_val_df.head(20)

,val,train,param
25,782.074884,588.680232,"{'max_depth': 10, 'n_estimators': 5}"
24,783.672066,703.780333,"{'max_depth': 10, 'n_estimators': 4}"
12,808.228044,590.887241,"{'max_depth': 5, 'n_estimators': 7}"
52,810.020077,706.111656,"{'max_depth': 30, 'n_estimators': 4}"
73,810.020077,706.111656,"{'max_depth': None, 'n_estimators': 4}"
59,810.020077,706.111656,"{'max_depth': 40, 'n_estimators': 4}"
66,810.020077,706.111656,"{'max_depth': 50, 'n_estimators': 4}"
45,811.289169,707.066157,"{'max_depth': 25, 'n_estimators': 4}"
26,812.870730,485.453336,"{'max_depth': 10, 'n_estimators': 7}"
39,816.610299,588.067928,"{'max_depth': 20, 'n_estimators': 5}"


The best model overfits.

In [ ]:
# make predictions
rf_yhat2 = m_rf2.predict(Xtrain)

# Calculate how much time it took to tune the hyperparameters and train the model
rf_duration2 = time.time() - rf_start2
print(f'The Random Forest model took {rf_duration2:.3f} seconds')

The Random Forest model took 34.552 seconds


In [ ]:
dump(m_rf2, 'Diss_models/rf2_59.joblib')

['Diss_models/rf2_59.joblib']

#### *Model 3*

In [ ]:
# specify the hyperparameters and their values
# n_estimators = number of decision trees used in the random forest
# max depth :  maximum number of levels in the trees
# min_samples_Split = Minimum number of samples required to split a node
rf3= RandomForestRegressor(random_state=20)
param_grid3 = [{'n_estimators': [1,2,3,4,5, 7, 10, 15], 
              'max_depth': [1,2,3,4,5,6,7,8,9,10,15,20,25,None]}]

tscv = TimeSeriesSplit(n_splits=5)

# start a new timer
rf_start3=time.time()

# we'll use tscv with 5 folds 
# we also want to get the train score for later
grid_search3 = GridSearchCV(estimator=rf3, cv=tscv,
                           param_grid=param_grid3,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# fit the estimator on the training set
grid_search3.fit(Xtrain, ytrain)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=20,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None

In [ ]:
# the best model's RMSE
rf_rmse_score3=np.sqrt(-grid_search3.best_score_)
print(f'The best Random Forest model has a RMSE of: {rf_rmse_score3}')

The best Random Forest model has a RMSE of: 782.0748840960143


The RMSE of this model is the same as the previous model's. 

In [ ]:
# the best random forest model
m_rf3 = grid_search3.best_estimator_
m_rf3

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
                      oob_score=False, random_state=20, verbose=0,
                      warm_start=False)

The best model has a max-depth of 8, 5 estimators and 2 min samples split.

In [ ]:
# training and validation RMSE

val_scores3 = grid_search3.cv_results_["mean_test_score"]
train_scores3 = grid_search3.cv_results_["mean_train_score"]
params3 = grid_search3.cv_results_["params"]

m_rf3_val_df=pd.DataFrame({'val': np.sqrt(-val_scores3),'train':  np.sqrt(-train_scores3), 'param': params3} )
m_rf3_val_df.sort_values('val', inplace=True)
m_rf3_val_df.head(20)

,val,train,param
76,782.074884,588.680232,"{'max_depth': 10, 'n_estimators': 5}"
75,783.672066,703.780333,"{'max_depth': 10, 'n_estimators': 4}"
37,808.228044,590.887241,"{'max_depth': 5, 'n_estimators': 7}"
107,810.020077,706.111656,"{'max_depth': None, 'n_estimators': 4}"
99,811.289169,707.066157,"{'max_depth': 25, 'n_estimators': 4}"
77,812.870730,485.453336,"{'max_depth': 10, 'n_estimators': 7}"
92,816.610299,588.067928,"{'max_depth': 20, 'n_estimators': 5}"
108,817.452398,589.979753,"{'max_depth': None, 'n_estimators': 5}"
69,817.768979,477.324686,"{'max_depth': 9, 'n_estimators': 7}"
100,818.365126,590.814984,"{'max_depth': 25, 'n_estimators': 5}"


It overfits

In [ ]:
# make predictions
rf_yhat3 = m_rf3.predict(Xtrain)

# Calculate how much time it took to tune the hyperparameters and train the model
rf_duration3 = time.time() - rf_start3
print(f'The Random Forest model took {rf_duration3:.3f} seconds')

The Random Forest model took 47.510 seconds


In [ ]:
dump(m_rf3, 'Diss_models/rf3_59.joblib')

['Diss_models/rf3_59.joblib']

#### 2.1.1.2 Without GT

#### *Model 1*

In [ ]:
# specify the hyperparameters and their values
# n_estimators = number of decision trees used in the random forest
# max depth :  maximum number of levels in the trees
# min_samples_Split = Minimum number of samples required to split a node
rf_lg= RandomForestRegressor(random_state=20)
param_grid_lg = [{'n_estimators': [3, 10, 20, 30, 40,50,100,120, 150,200], 
              'max_depth': [2, 4, 6, 8, 10, None]}]

tscv_lg = TimeSeriesSplit(n_splits=5)

# start a new timer
rf_start_lg=time.time()

# we'll use tscv with 5 folds 
# we also want to get the train score for later
grid_search_lg = GridSearchCV(estimator=rf_lg, cv=tscv_lg,
                           param_grid=param_grid_lg,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# fit the estimator on the training set
grid_search_lg.fit(Xtrain_light, ytrain_light)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=20,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None

In [ ]:
# the best model's RMSE
rf_rmse_score_lg=np.sqrt(-grid_search_lg.best_score_)
print(f'The best Random Forest model has a RMSE of: {rf_rmse_score_lg}')

The best Random Forest model has a RMSE of: 861.0886983592708


The RMSE of the model without GT is higher than the one with GT. So GT improves the accuracy of the forecasting model.

In [ ]:
# the best random forest model
rf_macro1 = grid_search_lg.best_estimator_
rf_macro1

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=3, n_jobs=None,
                      oob_score=False, random_state=20, verbose=0,
                      warm_start=False)

max_depth = 6, 3 estimators
Let's print the accuracy scores for every model evaluated during the grid search.

In [ ]:
# training and validation RMSE

val_scores_lg1 = grid_search_lg.cv_results_["mean_test_score"]
train_scores_lg1 = grid_search_lg.cv_results_["mean_train_score"]
params_lg1 = grid_search_lg.cv_results_["params"]

for val_score_lg, train_score_lg, param_lg in sorted(zip(val_scores_lg1, train_scores_lg1, params_lg1), reverse=True):
    print(np.sqrt(-val_score_lg), np.sqrt(-train_score_lg), param_lg)

861.0886983592708 912.4258003032517 {'max_depth': 10, 'n_estimators': 3}
862.9108491537247 621.2327937050684 {'max_depth': 6, 'n_estimators': 10}
882.8194218395797 910.7006542778795 {'max_depth': None, 'n_estimators': 3}
887.1084398547915 909.8178102305665 {'max_depth': 8, 'n_estimators': 3}
889.0132780046615 580.9045328034308 {'max_depth': 10, 'n_estimators': 10}
891.7753679354039 514.5511679300737 {'max_depth': 6, 'n_estimators': 20}
898.2872065709662 922.9118487884133 {'max_depth': 6, 'n_estimators': 3}
899.8160660002793 579.3916822787438 {'max_depth': None, 'n_estimators': 10}
903.2306097893096 595.0752614841458 {'max_depth': 8, 'n_estimators': 10}
913.8485050574081 468.3819249758976 {'max_depth': 6, 'n_estimators': 30}
922.8884590546244 463.79958867713884 {'max_depth': 6, 'n_estimators': 40}
931.1841262847162 477.4054335109545 {'max_depth': 6, 'n_estimators': 100}
931.307874903166 437.78172812415266 {'max_depth': 6, 'n_estimators': 200}
931.7000073001623 461.12486172598324 {'max_d

The RMSE for the validation set is higher than the RMSE of the training set. There may be some overfitting.

In [ ]:
# make predictions
rf_macro_yhat1 = rf_macro1.predict(Xtrain_light)

# Calculate how much time it took to tune the hyperparameters and train the model
rf_macro_time1 = time.time() - rf_start_lg
print(f'The Random Forest model took {rf_macro_time1:.3f} seconds')

The Random Forest model took 211.214 seconds


In [ ]:
dump(rf_macro1, 'Diss_models/macro_rf1_59.joblib')

['Diss_models/macro_rf1_59.joblib']

#### *Model 2*

In [ ]:
# specify the hyperparameters and their values
# n_estimators = number of decision trees used in the random forest
# max depth :  maximum number of levels in the trees
# min_samples_Split = Minimum number of samples required to split a node
rf_lg2= RandomForestRegressor(random_state=20)
param_grid_lg2 = [{'n_estimators': [1,3,5,7, 10,12, 15, 20,25, 30, 35], 
              'max_depth': [1,2,3, 4,5, 8, 10,15, None]}]

tscv_lg2 = TimeSeriesSplit(n_splits=5)

# start a new timer
rf_start_lg2=time.time()

# we'll use tscv with 5 folds 
# we also want to get the train score for later
grid_search_lg2 = GridSearchCV(estimator=rf_lg2, cv=tscv_lg2,
                           param_grid=param_grid_lg2,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# fit the estimator on the training set
grid_search_lg2.fit(Xtrain_light, ytrain_light)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=20,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None

In [ ]:
# the best model's RMSE
rf_rmse_score_lg2=np.sqrt(-grid_search_lg2.best_score_)
print(f'The best Random Forest model has a RMSE of: {rf_rmse_score_lg2}')

The best Random Forest model has a RMSE of: 812.8456146840018


The 1st model has a better RMSE. So we will keep the 1st one.

In [ ]:
# the best random forest model
rf_macro2 = grid_search_lg2.best_estimator_
rf_macro2

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
                      oob_score=False, random_state=20, verbose=0,
                      warm_start=False)

In [ ]:
# training and validation RMSE

val_scores_lg2 = grid_search_lg2.cv_results_["mean_test_score"]
train_scores_lg2 = grid_search_lg2.cv_results_["mean_train_score"]
params_lg2 = grid_search_lg2.cv_results_["params"]

for val_score_lg, train_score_lg, param_lg in sorted(zip(val_scores_lg2, train_scores_lg2, params_lg2)):
    print(np.sqrt(-val_score_lg), np.sqrt(-train_score_lg), param_lg)

3578.828546025996 4024.5237719798506 {'max_depth': 1, 'n_estimators': 3}
3543.2080899821426 4001.70144256181 {'max_depth': 1, 'n_estimators': 1}
3531.023811585649 3966.3987965931733 {'max_depth': 1, 'n_estimators': 35}
3530.3831280990275 3961.9095408584717 {'max_depth': 1, 'n_estimators': 15}
3523.6676756489987 3964.967686080891 {'max_depth': 1, 'n_estimators': 30}
3522.5083261298864 3968.042173924912 {'max_depth': 1, 'n_estimators': 12}
3521.5998802672416 3977.9157210347284 {'max_depth': 1, 'n_estimators': 7}
3521.1012884895367 3963.1982610448204 {'max_depth': 1, 'n_estimators': 25}
3520.6585713427185 3975.9229963387183 {'max_depth': 1, 'n_estimators': 10}
3520.4048034867733 3972.880152554972 {'max_depth': 1, 'n_estimators': 5}
3519.687518359633 3961.3912287860935 {'max_depth': 1, 'n_estimators': 20}
2140.7139707829024 1912.0006071375612 {'max_depth': 2, 'n_estimators': 3}
2139.8119495214073 1887.8511992515068 {'max_depth': 2, 'n_estimators': 1}
2134.9994807723006 1790.1777568446946 {

It overfits.

In [ ]:
# make predictions
rf_macro_yhat2 = rf_macro2.predict(Xtrain_light)

# Calculate how much time it took to tune the hyperparameters and train the model
rf_macro_time2 = time.time() - rf_start_lg2
print(f'The Random Forest model took {rf_macro_time2:.3f} seconds')

The Random Forest model took 62.966 seconds


In [ ]:
dump(rf_macro2, 'Diss_models/macro_rf2_59.joblib')

['Diss_models/macro_rf2_59.joblib']

### 2.1.2 **Gradient Boost**

Gradient Boosting is an ensemble of decision tree, similarly to RF. 

#### 2.1.2.1 *With GT*

#### Model 1

In [ ]:
param_gb1 = [{'n_estimators': [1, 10, 20, 30, 40,50,60,70,80,90,100,110,120,130,140,150], 
              'max_depth': [2, 4, 8, 10, 15, None]}]

gb_gt1= GradientBoostingRegressor(random_state=10)

tscv = TimeSeriesSplit(n_splits=5)

# start a new timer
gb_gt_start1=time.time()

# we'll use tscv with 5 folds 
# we also want to get the train score for later
gb_grid_search1 = GridSearchCV(estimator=gb_gt1, cv=tscv,
                           param_grid=param_gb1 ,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# fit the estimator on the training set
gb_grid_search1.fit(Xtrain, ytrain)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=GradientBoostingRegressor(alpha=0.9,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_we...
                                                 n_iter_no_change=None,
                                           

In [ ]:
# the best model's RMSE
gb_gt_rmse1=np.sqrt(-gb_grid_search1.best_score_)
print(f'The best Gradient Boosting model has a RMSE of: {gb_gt_rmse1}')

The best Gradient Boosting model has a RMSE of: 721.5282056409596


The RMSE is slightly better (lower) than the 1st model but higher than the two other RF models.

In [ ]:
# the best random forest model
gt_gb1 = gb_grid_search1.best_estimator_
gt_gb1

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=4,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=90,
                          n_iter_no_change=None, presort='auto',
                          random_state=10, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

Max depth: 4 and n estimators=140

In [ ]:
# training and validation RMSE
val_gt_gb1 = gb_grid_search1.cv_results_["mean_test_score"]
train_gt_gb1 = gb_grid_search1.cv_results_["mean_train_score"]
params_gt_gb1 = gb_grid_search1.cv_results_["params"]

for val_score, train_score, param in sorted(zip(val_gt_gb1, train_gt_gb1, params_gt_gb1)):
    print(np.sqrt(-val_score), np.sqrt(-train_score), param)

5747.123383626059 12287.312910306537 {'max_depth': 2, 'n_estimators': 1}
5696.424808696407 12265.104202250874 {'max_depth': 4, 'n_estimators': 1}
5676.655714001426 12262.04199522493 {'max_depth': 8, 'n_estimators': 1}
5675.066632461134 12261.843123530525 {'max_depth': 10, 'n_estimators': 1}
5672.312306086481 12261.76254718997 {'max_depth': 15, 'n_estimators': 1}
5671.620147027301 12261.741083014243 {'max_depth': None, 'n_estimators': 1}
2912.332778156556 4978.841004536571 {'max_depth': 2, 'n_estimators': 10}
2447.734569681877 4777.640133638891 {'max_depth': 4, 'n_estimators': 10}
2373.329559657073 4751.416361569173 {'max_depth': 10, 'n_estimators': 10}
2368.7745505022604 4750.653993480739 {'max_depth': 15, 'n_estimators': 10}
2353.598646389384 4750.449726372768 {'max_depth': None, 'n_estimators': 10}
2341.87042758548 4752.584513836471 {'max_depth': 8, 'n_estimators': 10}
1507.5457416894883 1968.4584683160392 {'max_depth': 2, 'n_estimators': 20}
1277.0747270242043 1658.5107239896956 {'m

It overfits a  lot

In [ ]:
# make predictions
gb_gt_yhat1 = gt_gb1.predict(Xtrain)

# Calculate how much time it took to tune the hyperparameters and train the model
gb_gt_duration1 = time.time() - gb_gt_start1
print(f'The Gradient Boosting model took {gb_gt_duration1:.3f} seconds')

The Gradient Boosting model took 811.262 seconds


In [ ]:
dump(gt_gb1, 'Diss_models/gt_gb1_59.joblib')

['Diss_models/gt_gb1_59.joblib']

#### Model 2

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

param_gb2 = [{'n_estimators': [50,75,100,125,150], 
              'max_depth': [1,2,3, 4, 5,10,15, None]}]

gb_gt2= GradientBoostingRegressor(random_state=15)

tscv = TimeSeriesSplit(n_splits=5)

# start a new timer
gb_gt_start2=time.time()

# we'll use tscv with 5 folds 
# we also want to get the train score for later
gb_grid_search2 = GridSearchCV(estimator=gb_gt2, cv=tscv,
                           param_grid=param_gb2 ,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# fit the estimator on the training set
gb_grid_search2.fit(Xtrain, ytrain)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=GradientBoostingRegressor(alpha=0.9,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_we...
                                                 n_estimators=100,
                                                

In [ ]:
# the best model's RMSE
gb_gt_rmse2=np.sqrt(-gb_grid_search2.best_score_)
print(f'The best Gradient Boosting model has a RMSE of: {gb_gt_rmse2}')

The best Gradient Boosting model has a RMSE of: 698.0760882575568


The RMSE is better than the last one and the RF's best one.

In [ ]:
# the best random forest model
gt_gb2 = gb_grid_search2.best_estimator_
gt_gb2

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=15, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
# training and validation RMSE
val_gt_gb2 = gb_grid_search2.cv_results_["mean_test_score"]
train_gt_gb2 = gb_grid_search2.cv_results_["mean_train_score"]
params_gt_gb2 = gb_grid_search2.cv_results_["params"]

for val_score, train_score, param in sorted(zip(val_gt_gb2, train_gt_gb2, params_gt_gb2)):
    print(np.sqrt(-val_score), np.sqrt(-train_score), param)

1107.0651367728606 1005.4885893363858 {'max_depth': 1, 'n_estimators': 50}
1080.432718071375 0.0018881254320189437 {'max_depth': None, 'n_estimators': 150}
1080.4318535056907 0.025980326631737924 {'max_depth': None, 'n_estimators': 125}
1080.4177882883137 0.3618767030894361 {'max_depth': None, 'n_estimators': 100}
1080.2256775340252 5.040781119417535 {'max_depth': None, 'n_estimators': 75}
1078.0165917040442 70.21584132468595 {'max_depth': None, 'n_estimators': 50}
1075.9020447896378 0.04403042811851251 {'max_depth': 15, 'n_estimators': 150}
1075.9007074180292 0.13855015474454832 {'max_depth': 15, 'n_estimators': 125}
1075.8698065478434 0.6509731394280973 {'max_depth': 15, 'n_estimators': 100}
1075.426417660987 6.183343641580003 {'max_depth': 15, 'n_estimators': 75}
1073.4283738718264 71.09100827499734 {'max_depth': 15, 'n_estimators': 50}
1047.9521566396606 4.39594212283365 {'max_depth': 10, 'n_estimators': 150}
1047.947941508637 6.982571199753738 {'max_depth': 10, 'n_estimators': 125

It overfits a lot

In [ ]:
# make predictions
gb_gt_yhat2 = gt_gb2.predict(Xtrain)

# Calculate how much time it took to tune the hyperparameters and train the model
gb_gt_duration2 = time.time() - gb_gt_start2
print(f'The Gradient Boosting model took {gb_gt_duration2:.3f} seconds')

The Gradient Boosting model took 308.935 seconds


In [ ]:
dump(gt_gb2, 'Diss_models/gt_gb2_59.joblib')

['Diss_models/gt_gb2_59.joblib']

#### 2.1.2.2 *Without GT*

#### Model 1

In [ ]:
param_gb1_mc =  [{'n_estimators': [100,125,150,175,200], 
              'max_depth': [2, 4, 8, 10, 15, None]}]

gb_mc1= GradientBoostingRegressor(random_state=10)

tscv = TimeSeriesSplit(n_splits=5)

# start a new timer
gb_mc_start1=time.time()

# we'll use tscv with 5 folds 
# we also want to get the train score for later
gb_mc_grids1 = GridSearchCV(estimator=gb_mc1, cv=tscv,
                           param_grid=param_gb1_mc  ,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# fit the estimator on the training set
gb_mc_grids1.fit(Xtrain_light, ytrain_light)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=GradientBoostingRegressor(alpha=0.9,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_we...
                                                 n_estimators=100,
                                                

In [ ]:
# the best model's RMSE
gb_mc_rmse1=np.sqrt(-gb_mc_grids1.best_score_)
print(f'The best Gradient Boosting model has a RMSE of: {gb_mc_rmse1}')

The best Gradient Boosting model has a RMSE of: 725.3575020607366


The RMSE is better than the best RF model without GT data.

In [ ]:
# the best Graduent Booster model
mc_gb1 = gb_mc_grids1.best_estimator_
mc_gb1

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=4,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=125,
                          n_iter_no_change=None, presort='auto',
                          random_state=10, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

Best model has a max_depth of 4 and 150 n_estimators.

In [ ]:
# training and validation RMSE
val_mc_gb1 = gb_mc_grids1.cv_results_["mean_test_score"]
train_mc_gb1 = gb_mc_grids1.cv_results_["mean_train_score"]
params_mc_gb1= gb_mc_grids1.cv_results_["params"]

for val_score, train_score, param in sorted(zip(val_mc_gb1, train_mc_gb1, params_mc_gb1)):
    print(np.sqrt(-val_score), np.sqrt(-train_score), param)

1116.0716644185125 0.06786173452748606 {'max_depth': 15, 'n_estimators': 150}
1116.0715399296557 0.03014570555306396 {'max_depth': 15, 'n_estimators': 175}
1116.071516182256 0.014498450227123998 {'max_depth': 15, 'n_estimators': 200}
1116.066574847931 0.21359389508141138 {'max_depth': 15, 'n_estimators': 125}
1116.0086470567594 0.9416930342055526 {'max_depth': 15, 'n_estimators': 100}
1113.2475298844322 0.001887994776239193 {'max_depth': None, 'n_estimators': 150}
1113.2475084804105 0.00031399460925033015 {'max_depth': None, 'n_estimators': 200}
1113.2475071485107 0.0003337179268622858 {'max_depth': None, 'n_estimators': 175}
1113.2463718181543 0.02598031885643525 {'max_depth': None, 'n_estimators': 125}
1113.227646692644 0.3618767011317362 {'max_depth': None, 'n_estimators': 100}
1081.432061376336 8.486831722278849 {'max_depth': 10, 'n_estimators': 125}
1081.4086368796263 13.864714700105843 {'max_depth': 10, 'n_estimators': 100}
1081.3931705088805 5.278949207156415 {'max_depth': 10, '

The best model overfits a lot with a training RMSE lower than the validation set.

Until 20 estimators, it seems to underfit.

In [ ]:
# make predictions
gb_mc_yhat1 = mc_gb1.predict(Xtrain_light)

# Calculate how much time it took to tune the hyperparameters and train the model
gb_mc_duration1 = time.time() - gb_mc_start1
print(f'The Gradient Boosting model took {gb_mc_duration1:.3f} seconds')

The Gradient Boosting model took 381.480 seconds


In [ ]:
dump(gb_mc_grids1, 'Diss_models/mc_gb1_59.joblib')

['Diss_models/mc_gb1_59.joblib']

#### Model 2

In [ ]:
param_gb2_mc =  [{'n_estimators': [1, 10,25,50,75,100,150,175,200], 
              'max_depth': [1,2, 4, 8, 10, 15,20, None]}]

gb_mc2= GradientBoostingRegressor(random_state=20)

tscv = TimeSeriesSplit(n_splits=5)

# start a new timer
gb_mc_start2=time.time()

# we'll use tscv with 5 folds 
# we also want to get the train score for later
gb_mc_grids2 = GridSearchCV(estimator=gb_mc2, cv=tscv,
                           param_grid=param_gb2_mc  ,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# fit the estimator on the training set
gb_mc_grids2.fit(Xtrain_light, ytrain_light)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=GradientBoostingRegressor(alpha=0.9,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_we...
                                                 n_iter_no_change=None,
                                           

In [ ]:
# the best model's RMSE
gb_mc_rmse2=np.sqrt(-gb_mc_grids2.best_score_)
print(f'The best Gradient Boosting model has a RMSE of: {gb_mc_rmse2}')

The best Gradient Boosting model has a RMSE of: 726.175684496707


Best RMSE without GT but higher than the best model with GT.

In [ ]:
# the best Graduent Booster model
mc_gb2 = gb_mc_grids2.best_estimator_
mc_gb2

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=4,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=150,
                          n_iter_no_change=None, presort='auto',
                          random_state=20, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
# training and validation RMSE
val_mc_gb2 = gb_mc_grids2.cv_results_["mean_test_score"]
train_mc_gb2 = gb_mc_grids2.cv_results_["mean_train_score"]
params_mc_gb2= gb_mc_grids2.cv_results_["params"]

for val_score, train_score, param in sorted(zip(val_mc_gb2, train_mc_gb2, params_mc_gb2)):
    print(np.sqrt(-val_score), np.sqrt(-train_score), param)

5953.013689740608 12384.87607711152 {'max_depth': 1, 'n_estimators': 1}
5747.123383626059 12287.312910306537 {'max_depth': 2, 'n_estimators': 1}
5696.424808696407 12265.104202250874 {'max_depth': 4, 'n_estimators': 1}
5674.310476577559 12261.843226379413 {'max_depth': 10, 'n_estimators': 1}
5669.410742983671 12262.043259960148 {'max_depth': 8, 'n_estimators': 1}
5669.409661247471 12261.765849743331 {'max_depth': 15, 'n_estimators': 1}
5668.935966825466 12261.741083014243 {'max_depth': None, 'n_estimators': 1}
5668.849237533715 12261.74838829801 {'max_depth': 20, 'n_estimators': 1}
3915.2631684193702 5970.5292760663315 {'max_depth': 1, 'n_estimators': 10}
2912.332778156556 4978.84100453657 {'max_depth': 2, 'n_estimators': 10}
2448.618095278028 4777.640133638891 {'max_depth': 4, 'n_estimators': 10}
2361.188168618521 4750.512775952635 {'max_depth': 20, 'n_estimators': 10}
2354.605233136267 4750.449726372768 {'max_depth': None, 'n_estimators': 10}
2354.4727265158435 4751.428931943747 {'max

It overfits

In [ ]:
# make predictions
gb_mc_yhat2 = mc_gb2.predict(Xtrain_light)

# Calculate how much time it took to tune the hyperparameters and train the model
gb_mc_duration2 = time.time() - gb_mc_start2
print(f'The Gradient Boosting model took {gb_mc_duration2:.3f} seconds')

The Gradient Boosting model took 245.071 seconds


In [ ]:
dump(gb_mc_grids2, 'Diss_models/mc_gb2_59.joblib')

['Diss_models/mc_gb2_59.joblib']

### 2.1.3 Adaboost

#### 2.1.3.1 With GT

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

adb_param_gt1 = {'n_estimators': [3, 5,10,15, 20, 50], 'learning_rate': [0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1,1.25],
     'loss' : ['linear', 'square', 'exponential']}

adb_reg=AdaBoostRegressor(random_state=10)

tscv = TimeSeriesSplit(n_splits=5)

#start a new timer
adb_st_gt1 = time.time()

# we'll use tscv with 5 folds 
# we also want to get the train score for later
adb_grid_gt = GridSearchCV(estimator=adb_reg, cv=tscv,
                           param_grid=adb_param_gt1 ,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

#fit the best model and hyperparameters to the training dataset
adb_grid_gt.fit(Xtrain, ytrain)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=AdaBoostRegressor(base_estimator=None, learning_rate=1.0,
                                         loss='linear', n_estimators=50,
                                         random_state=10),
             iid='warn', n_jobs=None,
             param_grid={'learning_rate': [0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1,
                                           1.25],
                         'loss': ['linear', 'square', 'exponential'],
                         'n_estimators': [3, 5, 10, 15, 20, 50]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='neg_mean_squared_error', verbose=0)

In [ ]:
#RMSE score
adb_gt_rmse = np.sqrt(-adb_grid_gt.best_score_)
print(f'The best Adaboost model has a RMSE of: {adb_gt_rmse}')

The best Adaboost model has a RMSE of: 1213.4565186811283


In [ ]:
# the best model
adb_gt_1 = adb_grid_gt.best_estimator_
adb_gt_1

AdaBoostRegressor(base_estimator=None, learning_rate=0.75, loss='exponential',
                  n_estimators=5, random_state=10)

In [ ]:
# training and validation RMSE
adb_gt_val = adb_grid_gt.cv_results_["mean_test_score"]
adb_gt_train = adb_grid_gt.cv_results_["mean_train_score"]
adb_gt_params = adb_grid_gt.cv_results_["params"]

for adb_val_score, adb_train_score, adb_param in sorted(zip(adb_gt_val, adb_gt_train, adb_gt_params)):
    print(np.sqrt(-adb_val_score), np.sqrt(-adb_train_score), adb_param )

2696.975892578078 1683.0609847351725 {'learning_rate': 1.25, 'loss': 'square', 'n_estimators': 50}
2630.2638799514016 1702.166071870767 {'learning_rate': 0.75, 'loss': 'square', 'n_estimators': 50}
2543.16482907501 1422.0041159268285 {'learning_rate': 1, 'loss': 'square', 'n_estimators': 50}
2536.158927322889 1590.684689787661 {'learning_rate': 0.5, 'loss': 'square', 'n_estimators': 50}
2510.4373940754504 1329.790356823805 {'learning_rate': 1.25, 'loss': 'exponential', 'n_estimators': 50}
2508.9370695480543 1371.082835770868 {'learning_rate': 1.25, 'loss': 'square', 'n_estimators': 20}
2498.825618010646 1314.4601733468696 {'learning_rate': 1, 'loss': 'exponential', 'n_estimators': 50}
2474.570017543408 1387.331293520457 {'learning_rate': 1.25, 'loss': 'square', 'n_estimators': 15}
2428.897019619022 1097.143653031451 {'learning_rate': 0.25, 'loss': 'square', 'n_estimators': 50}
2428.313015742398 1239.4686147106079 {'learning_rate': 1, 'loss': 'square', 'n_estimators': 15}
2411.257889491

It overfits.

In [ ]:
# make predictions
adb_gt_yhat = adb_gt_1.predict(Xtrain)

# Calculate how much time it took to tune the hyperparameters and train the model
adb_dur_gt = time.time() - adb_st_gt1 
print(f'The Adaboost model took {adb_dur_gt:.3f} seconds')

The Adaboost model took 43.678 seconds


In [ ]:
dump(adb_gt_1, 'Diss_models/adb_gt_1_59.joblib')

['Diss_models/adb_gt_1_59.joblib']

#### 2.1.3.2 Without GT

In [ ]:
from sklearn.ensemble import AdaBoostRegressor

adb_param_mc = {'n_estimators': [3, 10, 20, 50], 'learning_rate': [0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1,1.25],
     'loss' : ['linear', 'square', 'exponential']}

adb_reg=AdaBoostRegressor(random_state=8)

tscv = TimeSeriesSplit(n_splits=5)

#start a new timer
adb_st_mc = time.time()

# we'll use tscv with 5 folds 
# we also want to get the train score for later
adb_grid_mc = GridSearchCV(estimator=adb_reg, cv=tscv,
                           param_grid=adb_param_mc ,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

#fit the best model and hyperparameters to the training dataset
adb_grid_mc.fit(Xtrain_light, ytrain_light)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=AdaBoostRegressor(base_estimator=None, learning_rate=1.0,
                                         loss='linear', n_estimators=50,
                                         random_state=8),
             iid='warn', n_jobs=None,
             param_grid={'learning_rate': [0.001, 0.01, 0.1, 0.25, 0.5, 0.75, 1,
                                           1.25],
                         'loss': ['linear', 'square', 'exponential'],
                         'n_estimators': [3, 10, 20, 50]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='neg_mean_squared_error', verbose=0)

In [ ]:
#RMSE score
adb_mc_rmse = np.sqrt(-adb_grid_mc.best_score_)
print(f'The best Adaboost model has a RMSE of: {adb_mc_rmse}')

The best Adaboost model has a RMSE of: 1234.1913271324363


In [ ]:
# the best model
adb_mc = adb_grid_mc.best_estimator_
adb_mc

AdaBoostRegressor(base_estimator=None, learning_rate=0.5, loss='exponential',
                  n_estimators=20, random_state=8)

In [ ]:
# training and validation RMSE
adb_mc_val = adb_grid_mc.cv_results_["mean_test_score"]
adb_mc_train = adb_grid_mc.cv_results_["mean_train_score"]
adb_mc_params = adb_grid_mc.cv_results_["params"]

for adb_val_score, adb_train_score, adb_param in sorted(zip(adb_mc_val, adb_mc_train, adb_mc_params)):
    print(np.sqrt(-adb_val_score), np.sqrt(-adb_train_score), adb_param)

2809.007495722028 1753.7989347592372 {'learning_rate': 1, 'loss': 'square', 'n_estimators': 50}
2760.066981440683 1650.962658640144 {'learning_rate': 0.75, 'loss': 'square', 'n_estimators': 50}
2759.2144926649257 1655.9479888488488 {'learning_rate': 1.25, 'loss': 'square', 'n_estimators': 50}
2628.70994659514 1553.3697590795232 {'learning_rate': 1, 'loss': 'square', 'n_estimators': 20}
2550.9165682818134 1588.204312008094 {'learning_rate': 0.5, 'loss': 'square', 'n_estimators': 50}
2515.2337250070696 1435.459940235373 {'learning_rate': 1.25, 'loss': 'square', 'n_estimators': 20}
2483.2631156627767 1320.8212463252682 {'learning_rate': 1.25, 'loss': 'exponential', 'n_estimators': 50}
2475.3510500653065 1275.3970523449273 {'learning_rate': 1, 'loss': 'exponential', 'n_estimators': 50}
2423.9612000045245 1148.4462850901841 {'learning_rate': 0.25, 'loss': 'square', 'n_estimators': 50}
2422.745849508738 1257.4319610318616 {'learning_rate': 1.25, 'loss': 'square', 'n_estimators': 10}
2379.054

It overfits

In [ ]:
# make predictions
adb_mc_yhat = adb_mc.predict(Xtrain_light)

# Calculate how much time it took to tune the hyperparameters and train the model
adb_dur_mc = time.time() - adb_st_mc
print(f'The Adaboost model took {adb_dur_mc:.3f} seconds')

The Adaboost model took 309.305 seconds


In [ ]:
dump(adb_mc, 'Diss_models/adb_mc_59.joblib')

NameError: name 'dump' is not defined

### 2.1.4 Support Vector

#### 2.1.4.1 **With GT**

In [ ]:
from sklearn.svm import SVR

svr_param_grid = [
    {'C': [1000,2000,3000,4000,5000,6000,10000,15000,20000], 
     'gamma': ["scale", "auto", 0.001, 0.01, 0.1, 1, 3, 5, 10]
    },
]


svr = SVR(kernel="rbf")

tscv = TimeSeriesSplit(n_splits=5)

#start a new timer
svr_st_gt1 = time.time()

# we'll use tscv with 5 folds 
# we also want to get the train score for later
svr_grid_gt = GridSearchCV(estimator=svr , cv=tscv,
                           param_grid=svr_param_grid  ,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

#fit the best model and hyperparameters to the training dataset
svr_grid_gt.fit(Xtrain, ytrain)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [1000, 2000, 3000, 4000, 5000, 6000, 10000,
                                15000, 20000],
                          'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1, 3, 5,
                                    10]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='neg_mean_squared_error', verbose=0)

In [ ]:
#RMSE score
svr_gt_rmse = np.sqrt(-svr_grid_gt.best_score_)
print(f'The best SVR model has a RMSE of: {svr_gt_rmse}')

The best SVR model has a RMSE of: 580.8140097001826


In [ ]:
# the best model
svr_gt_1 = svr_grid_gt.best_estimator_
svr_gt_1

SVR(C=20000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.001,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
# training and validation RMSE
svr_gt_val = svr_grid_gt.cv_results_["mean_test_score"]
svr_gt_train = svr_grid_gt.cv_results_["mean_train_score"]
svr_gt_params = svr_grid_gt.cv_results_["params"]

for svr_val_score, svr_train_score, svr_param in sorted(zip(svr_gt_val, svr_gt_train, svr_gt_params), reverse=True):
    print(np.sqrt(-svr_val_score), np.sqrt(-svr_train_score), svr_param )

580.8140097001826 1813.2355307389723 {'C': 20000, 'gamma': 0.001}
583.770974837227 1822.357282896512 {'C': 15000, 'gamma': 0.001}
588.7098078492368 1837.8770172814352 {'C': 10000, 'gamma': 0.001}
670.0547016811443 2512.607753094207 {'C': 6000, 'gamma': 0.001}
698.8644426359577 1895.7351276978839 {'C': 15000, 'gamma': 0.01}
700.9256271661445 2037.2053083969352 {'C': 10000, 'gamma': 0.01}
711.9579218298184 1800.1529260061511 {'C': 20000, 'gamma': 0.01}
716.9036166555857 2223.4794419461596 {'C': 6000, 'gamma': 0.01}
741.9811027068546 2274.6160718528736 {'C': 5000, 'gamma': 0.01}
789.886555856874 3586.4200153275838 {'C': 5000, 'gamma': 0.001}
790.7713180390426 2350.7965073246237 {'C': 4000, 'gamma': 0.01}
822.6267276457143 2145.216093960729 {'C': 10000, 'gamma': 'auto'}
823.1303610137886 1907.2701515765339 {'C': 15000, 'gamma': 'auto'}
834.0994472524537 2487.5405426289167 {'C': 3000, 'gamma': 0.01}
842.5824919194312 1759.7762513817072 {'C': 20000, 'gamma': 'auto'}
849.6340805479635 2449.13

It seems to underfit

In [ ]:
# make predictions
svr_gt_yhat = svr_gt_1.predict(Xtrain)

# Calculate how much time it took to tune the hyperparameters and train the model
svr_dur_gt = time.time() - svr_st_gt1 
print(f'The best SVR model took {svr_dur_gt:.3f} seconds')

The best SVR model took 146.876 seconds


In [ ]:
dump(svr_gt_1, 'Diss_models/svr_gt_1_59.joblib')

['Diss_models/svr_gt_1_59.joblib']

#### 2.1.4.2 **Without GT**

In [ ]:
svr_param_mc = [
    {'C': [2500,5000,10000,15000,20000,25000,30000,40000,50000], 
     'gamma': ["scale", "auto", 0.001, 0.01, 0.1, 1, 3, 5, 10]
    },
]

svr = SVR(kernel="rbf")
tscv = TimeSeriesSplit(n_splits=5)

#start a new timer
svr_st_mc = time.time()

# we'll use tscv with 5 folds 
# we also want to get the train score for later
svr_grid_mc = GridSearchCV(estimator=svr , cv=tscv,
                           param_grid=svr_param_mc  ,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

#fit the best model and hyperparameters to the training dataset
svr_grid_mc.fit(Xtrain_light, ytrain_light)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [2500, 5000, 10000, 15000, 20000, 25000, 30000,
                                40000, 50000],
                          'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1, 3, 5,
                                    10]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='neg_mean_squared_error', verbose=0)

In [ ]:
#RMSE score
svr_mc_rmse = np.sqrt(-svr_grid_mc.best_score_)
print(f'The best SVR model has a RMSE of: {svr_mc_rmse}')

The best SVR model has a RMSE of: 575.3617566288362


In [ ]:
# the best model
svr_mc_1 = svr_grid_mc.best_estimator_
svr_mc_1

SVR(C=50000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.001,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
# training and validation RMSE
svr_mc_val = svr_grid_mc.cv_results_["mean_test_score"]
svr_mc_train = svr_grid_mc.cv_results_["mean_train_score"]
svr_mc_params = svr_grid_mc.cv_results_["params"]

for svr_val_score, svr_train_score, svr_param in sorted(zip(svr_mc_val, svr_mc_train, svr_mc_params), reverse=True):
    print(np.sqrt(-svr_val_score), np.sqrt(-svr_train_score), svr_param )

575.3617566288362 1785.993848032642 {'C': 50000, 'gamma': 0.001}
576.6069683536357 1796.4880487455257 {'C': 40000, 'gamma': 0.001}
577.638831841501 1808.1873686039 {'C': 30000, 'gamma': 0.001}
578.6044202696359 1814.3614170694307 {'C': 25000, 'gamma': 0.001}
580.6971318989368 1821.7859981297245 {'C': 20000, 'gamma': 0.001}
582.552664046248 1830.4776571208358 {'C': 15000, 'gamma': 0.001}
588.1347882501108 1846.720153749813 {'C': 10000, 'gamma': 0.001}
709.8424102313438 2003.8912174057384 {'C': 10000, 'gamma': 0.01}
733.3101885678426 2257.118683883096 {'C': 5000, 'gamma': 0.01}
740.0041273498119 1836.1779397470905 {'C': 15000, 'gamma': 0.01}
760.6697114262232 1765.8350294050015 {'C': 20000, 'gamma': 0.01}
769.9284310631878 3497.0265037736867 {'C': 5000, 'gamma': 0.001}
778.5269319047584 1726.1251279479475 {'C': 25000, 'gamma': 0.01}
796.301943875678 1700.1399250228262 {'C': 30000, 'gamma': 0.01}
802.1893593951331 2008.1621514713295 {'C': 10000, 'gamma': 'auto'}
814.6405741070832 2607.704

The best model underfits.

In [ ]:
# make predictions
svr_mc_yhat = svr_mc_1.predict(Xtrain_light)

# Calculate how much time it took to tune the hyperparameters and train the model
svr_dur_mc = time.time() - svr_st_mc
print(f'The best SVR model took {svr_dur_mc:.3f} seconds')

The best SVR model took 187.301 seconds


In [ ]:
dump(svr_mc_1, 'Diss_models/svr_mc_1_59.joblib')

['Diss_models/svr_mc_1_59.joblib']

### 2.1.5 RMSE

In [ ]:
# Load the models if they took a lot of time to train
from joblib import dump
from joblib import load

rf_gt=load('Diss_models/rf3.joblib')
rf_mc = load('Diss_models/macro_rf1.joblib')
gb_gt=load('Diss_models/gt_gb2.joblib')
gb_mc=load('Diss_models/mc_gb2.joblib')


In [ ]:
rmse_persi_all

4936.789529657899

In [ ]:
RMSE= {
    'RF With GT 1': [rf_rmse_score],
    'RF With GT 2': [rf_rmse_score2],
    'RF With GT 3': [rf_rmse_score3],
    'RF macro 1': [rf_rmse_score_lg],
    'RF macro 2': [rf_rmse_score_lg2],
    'GB with GT 1': [gb_gt_rmse1],
    'GB with GT 2': [gb_gt_rmse2],
    'GB macro 1': [gb_mc_rmse1],
    'GB macro 2': [gb_mc_rmse2],
    'ADB With GT': [adb_gt_rmse],
    'ADB macro': [adb_mc_rmse],
    'SVR with GT':[svr_gt_rmse],
    'SVR macro':[svr_mc_rmse]
}

scores_df=pd.DataFrame(RMSE).transpose()
scores_df.columns=['RMSE']
scores_df["Vs Baseline"] =(scores_df["RMSE"]/rmse_persi_all-1)*100
scores_df

,RMSE,Vs Baseline
RF With GT 1,782.074884,-84.158229
RF With GT 2,782.074884,-84.158229
RF With GT 3,782.074884,-84.158229
RF macro 1,861.088698,-82.557719
RF macro 2,812.845615,-83.534935
GB with GT 1,721.528206,-85.384667
GB with GT 2,698.076088,-85.859715
GB macro 1,725.357502,-85.307101
GB macro 2,726.175684,-85.290528
ADB With GT,1213.456519,-75.420129


Best models are the SVR and GB in terms of RMSE.

* RF: the GT model has a better RMSE than the one without GT data.
* GB: The model without GT slightly beats the one with GT data.
* ADB: The model with GT data is more accurate than the one without GT.
* SVR: the model without GT data is better than the one with.

### 2.1.6  Features importance

#### 2.1.6.1 RF

**Random Forest with GT**

In [ ]:
rf_gt_fi = m_rf3.feature_importances_
rf_gt_df = pd.DataFrame(rf_gt_fi, Xtrain.columns)
rf_gt_df.rename(columns={0:'Feature importance'}, inplace=True) 
rf_gt_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.829594
CS_lag1,0.154538
CS_lag2,0.012466
Restaurants: 276,0.000777
cci_lag1,0.000418
cci,0.000324
cci_lag2,0.000307
DI,0.000304
Apparel: 68,0.000242
DI_lag1,0.000229


**Random forest without GT data**

In [ ]:
rf_mc_fi = rf_macro2.feature_importances_
rf_mc_df = pd.DataFrame(rf_mc_fi, Xtrain_light.columns)
rf_mc_df.rename(columns={0:'Feature importance'}, inplace=True) 
rf_mc_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.829595
CS_lag1,0.160717
CS_lag2,0.006274
Restaurants: 276,0.000777
cci,0.000490
cci_lag1,0.000368
DI,0.000342
Apparel: 68,0.000242
cci_lag2,0.000238
DI_lag1,0.000226


#### 2.1.6.2 GB

**Gradient Boosting model with GT**

In [ ]:
gb_gt_fi = gt_gb2.feature_importances_
gb_gt_df = pd.DataFrame(gb_gt_fi, Xtrain.columns)
gb_gt_df.rename(columns={0:'Feature importance'}, inplace=True) 
gb_gt_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.806302
CS_lag1,0.186479
CS_lag2,0.004093
Restaurants: 276,0.000964
DI_lag2,0.000371
cci,0.000363
cci_lag2,0.000307
Apparel: 68,0.000264
3m Bond,0.000163
cci_lag1,0.000136


**Gradient Boosting with only macrovariables**

In [ ]:
gb_mc_fi = mc_gb2.feature_importances_
gb_mc_df = pd.DataFrame(gb_mc_fi , Xtrain_light.columns)
gb_mc_df.rename(columns={0:'Feature importance'}, inplace=True) 
gb_mc_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.834073
CS_lag1,0.152731
CS_lag2,0.009581
Restaurants: 276,0.000973
cci_lag1,0.000446
DI_lag1,0.000386
Apparel: 68,0.000378
DI,0.000309
Automotive Industry: 1190,0.000235
cci,0.000160


#### 2.1.6.3 AdaBoost

**AdaBoost with GT**

In [ ]:
adb_gt_fi = adb_gt_1.feature_importances_
adb_gt_df = pd.DataFrame(adb_gt_fi, Xtrain.columns)
adb_gt_df.rename(columns={0:'Feature importance'}, inplace=True) 
adb_gt_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.851604
CS_lag1,0.134580
CS_lag2,0.010075
Restaurants: 276,0.002378
cci_lag1,0.000601
DI,0.000269
cci_lag2,0.000268
DI_lag1,0.000225
Health Insurance: 249,0.000000
Hobbies & Leisure: 65,0.000000


**AdaBoost without GT data**

In [ ]:
adb_mc_fi = adb_mc.feature_importances_
adb_mc_df = pd.DataFrame(adb_mc_fi, Xtrain_light.columns)
adb_mc_df.rename(columns={0:'Feature importance'}, inplace=True) 
adb_mc_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.745111
CS_lag1,0.211348
Restaurants: 276,0.029203
CS_lag2,0.006899
Apparel: 68,0.006301
DI,0.000373
cci_lag1,0.000231
cci_lag2,0.000157
cci,0.000140
DI_lag2,0.000086


## 2.2 Models with only lagged values of CS, macroeconomic variables and all GT variables

Creation of 2 dataframes without the actcual values of the macroeconomic variables, since in real life we would be available later than the GT index.

In [ ]:
Xtrain_lag=Xtrain.drop(['DI', '3m Bond', 'cci', 'CPI', 'vix'], axis=1)
Xtrain_lag.head()

,GT,GT_lag1,GT_lag2,DI_lag1,DI_lag2,3m Bond_lag1,3m Bond_lag2,cci_lag1,cci_lag2,CPI_lag1,...,Restaurants: 276,Small Kitchen Appliances: 1292,Sporting Goods: 263,Tobacco Products: 123,Transportation & Logistics: 50,Vehicle Fuels & Lubricants: 1269,Vehicle Maintenance: 138,Vehicle Parts & Accessories: 89,Water Supply & Treatment: 1349,Wine: 405
quarter,,,,,,,,,,,,,,,,,,,,,
2005Q1,2.525412,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
2005Q2,-0.367123,2.525412,0.000000,0.286653,0.000000,0.332819,0.000000,0.954549,0.000000,-0.620977,...,0,0,0,0,0,0,0,0,0,0
2005Q3,0.818342,-0.367123,2.525412,1.226200,0.286653,0.199119,0.332819,0.822608,0.954549,0.391964,...,0,0,0,0,0,0,0,0,0,0
2005Q4,-2.927727,0.818342,-0.367123,0.756426,1.226200,-0.373748,0.199119,0.818727,0.822608,-0.215800,...,0,0,0,0,0,0,0,0,0,0
2006Q1,1.908970,-2.927727,0.818342,1.312595,0.756426,0.032748,-0.373748,0.562606,0.818727,-0.215800,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Copy without the GT data
Xtrain_light_lag=Xtrain_light.copy().drop(['DI', '3m Bond', 'cci', 'CPI', 'vix'], axis=1)
Xtrain_light_lag.head()

,DI_lag1,DI_lag2,3m Bond_lag1,3m Bond_lag2,cci_lag1,cci_lag2,CPI_lag1,CPI_lag2,vix_lag1,vix_lag2,...,Restaurants: 276,Small Kitchen Appliances: 1292,Sporting Goods: 263,Tobacco Products: 123,Transportation & Logistics: 50,Vehicle Fuels & Lubricants: 1269,Vehicle Maintenance: 138,Vehicle Parts & Accessories: 89,Water Supply & Treatment: 1349,Wine: 405
quarter,,,,,,,,,,,,,,,,,,,,,
2005Q1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0,0,0,0,0,0,0,0,0,0
2005Q2,0.286653,0.000000,0.332819,0.000000,0.954549,0.000000,-0.620977,0.000000,-0.352044,0.000000,...,0,0,0,0,0,0,0,0,0,0
2005Q3,1.226200,0.286653,0.199119,0.332819,0.822608,0.954549,0.391964,-0.620977,0.075050,-0.352044,...,0,0,0,0,0,0,0,0,0,0
2005Q4,0.756426,1.226200,-0.373748,0.199119,0.818727,0.822608,-0.215800,0.391964,-0.122374,0.075050,...,0,0,0,0,0,0,0,0,0,0
2006Q1,1.312595,0.756426,0.032748,-0.373748,0.562606,0.818727,-0.215800,-0.215800,0.378424,-0.122374,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
Xtest_lag=Xtest.drop(['DI', '3m Bond', 'cci', 'CPI', 'vix'], axis=1)
# Copy without the GT data
Xtest_light_lag=Xtest_light.copy().drop(['DI', '3m Bond', 'cci', 'CPI', 'vix'], axis=1)

### 2.2.1 RF

**With GT**

In [ ]:
rf_gt_lag = RandomForestRegressor(random_state=20)
parg_rf_gt_lag = [{'n_estimators': [1,2,3,4,5,6, 7, 10, 15], 
              'max_depth': [1,2,3,4,5,6,7,8,9,10,15,20,25,None]}]

tscv = TimeSeriesSplit(n_splits=5)

# we'll use tscv with 5 folds 
# we also want to get the train score for later
grid_rf_gt_lag = GridSearchCV(estimator=rf_gt_lag, cv=tscv,
                           param_grid=parg_rf_gt_lag,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# fit the estimator on the training set
grid_rf_gt_lag.fit(Xtrain_lag, ytrain)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=20,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None

In [ ]:
# the best model's RMSE
rf_GT_lag_rmse=np.sqrt(-grid_rf_gt_lag.best_score_)
print(f'The best Random Forest model has a RMSE of: {rf_GT_lag_rmse}')

The best Random Forest model has a RMSE of: 768.0451967514052


In [ ]:
# the best random forest model
rf_gt_lag= grid_rf_gt_lag.best_estimator_
rf_gt_lag

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=9,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=None,
                      oob_score=False, random_state=20, verbose=0,
                      warm_start=False)

In [ ]:
val_rf_gt_lag = grid_rf_gt_lag.cv_results_["mean_test_score"]
train_rf_gt_lag = grid_rf_gt_lag.cv_results_["mean_train_score"]
par_rf_gt_lag = grid_rf_gt_lag.cv_results_["params"]

rf_gt_lag_val_df=pd.DataFrame({'val': np.sqrt(-val_rf_gt_lag),'train':  np.sqrt(-train_rf_gt_lag), 'param': par_rf_gt_lag} )
rf_gt_lag_val_df.sort_values('val', inplace=True)
rf_gt_lag_val_df.head(20)

,val,train,param
76,768.045197,618.159699,"{'max_depth': 9, 'n_estimators': 5}"
67,768.086086,624.296247,"{'max_depth': 8, 'n_estimators': 5}"
66,773.027828,746.652475,"{'max_depth': 8, 'n_estimators': 4}"
75,777.740429,738.255276,"{'max_depth': 9, 'n_estimators': 4}"
105,783.596391,636.208413,"{'max_depth': 20, 'n_estimators': 7}"
94,785.568141,621.594327,"{'max_depth': 15, 'n_estimators': 5}"
93,786.209312,742.760052,"{'max_depth': 15, 'n_estimators': 4}"
69,788.754894,638.537335,"{'max_depth': 8, 'n_estimators': 7}"
87,790.658726,637.176852,"{'max_depth': 10, 'n_estimators': 7}"
103,791.409924,619.972144,"{'max_depth': 20, 'n_estimators': 5}"


It overfits.

In [ ]:
# make predictions
rf_gt_lag_train_yhat =rf_gt_lag.predict(Xtrain_lag)

In [ ]:
dump(rf_gt_lag, 'Diss_models/rf_gt_lag_59.joblib')

['Diss_models/rf_gt_lag_59.joblib']

**Without GT**

In [ ]:
rf_mc_lag = RandomForestRegressor(random_state=20)
parg_rf_mc_lag = [{'n_estimators': [1,2,3,4,5,6, 7, 10, 15], 
              'max_depth': [1,2,3,4,5,6,7,8,9,10,15,20,25,None]}]

tscv = TimeSeriesSplit(n_splits=5)

# we'll use tscv with 5 folds 
# we also want to get the train score for later
grid_rf_mc_lag = GridSearchCV(estimator=rf_mc_lag, cv=tscv,
                           param_grid=parg_rf_mc_lag,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# fit the estimator on the training set
grid_rf_mc_lag.fit(Xtrain_light_lag, ytrain)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=20,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None

In [ ]:
# the best model's RMSE
rf_mc_lag_rmse=np.sqrt(-grid_rf_mc_lag.best_score_)
print(f'The best Random Forest model has a RMSE of: {rf_mc_lag_rmse}')

The best Random Forest model has a RMSE of: 787.3571320357379


In [ ]:
# the best random forest model
rf_mc_lag= grid_rf_mc_lag.best_estimator_
rf_mc_lag

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=8,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=7, n_jobs=None,
                      oob_score=False, random_state=20, verbose=0,
                      warm_start=False)

In [ ]:
val_rf_mc_lag = grid_rf_mc_lag.cv_results_["mean_test_score"]
train_rf_mc_lag = grid_rf_mc_lag.cv_results_["mean_train_score"]
par_rf_mc_lag = grid_rf_mc_lag.cv_results_["params"]

rf_mc_lag_val_df=pd.DataFrame({'val': np.sqrt(-val_rf_mc_lag),'train':  np.sqrt(-train_rf_mc_lag), 'param': par_rf_mc_lag} )
rf_mc_lag_val_df.sort_values('val', inplace=True)
rf_mc_lag_val_df.head(20)

,val,train,param
69,787.357132,451.610125,"{'max_depth': 8, 'n_estimators': 7}"
67,788.193124,429.838009,"{'max_depth': 8, 'n_estimators': 5}"
49,792.810498,477.575759,"{'max_depth': 6, 'n_estimators': 5}"
50,793.627097,535.821057,"{'max_depth': 6, 'n_estimators': 6}"
84,793.704983,449.592109,"{'max_depth': 10, 'n_estimators': 4}"
66,793.733708,482.898286,"{'max_depth': 8, 'n_estimators': 4}"
51,795.094749,496.651382,"{'max_depth': 6, 'n_estimators': 7}"
85,795.215549,401.509736,"{'max_depth': 10, 'n_estimators': 5}"
102,797.171129,471.839545,"{'max_depth': 20, 'n_estimators': 4}"
48,800.803871,527.552183,"{'max_depth': 6, 'n_estimators': 4}"


It overfits.

In [ ]:
# make predictions
rf_mc_lag_train_yhat =rf_mc_lag.predict(Xtrain_light_lag)

In [ ]:
dump(rf_mc_lag, 'Diss_models/rf_mc_lag_59.joblib')

['Diss_models/rf_mc_lag_59.joblib']

### 2.2.2 Gradient Boost

**With GT**

In [ ]:
parg_gb_gt_lag = [{'n_estimators': [1, 10, 20, 30, 40,50,60,70,80,90,100,110,120,130,140,150], 
              'max_depth': [2, 3,4,5, 8, 10, 15, None]}]

gb_gt_lag= GradientBoostingRegressor(random_state=10)

tscv = TimeSeriesSplit(n_splits=5)

# we'll use tscv with 5 folds 
# we also want to get the train score for later
gb_gt_lag_grid = GridSearchCV(estimator=gb_gt_lag, cv=tscv,
                           param_grid=parg_gb_gt_lag ,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# fit the estimator on the training set
gb_gt_lag_grid.fit(Xtrain_lag, ytrain)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=GradientBoostingRegressor(alpha=0.9,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_we...
                                                 presort='auto',
                                                 r

In [ ]:
# the best model's RMSE
gb_GT_lag_rmse=np.sqrt(-gb_gt_lag_grid.best_score_)
print(f'The best Gradient Boosting model has a RMSE of: {gb_GT_lag_rmse}')

The best Gradient Boosting model has a RMSE of: 702.2154331897921


In [ ]:
# the best random forest model
gb_gt_lag= gb_gt_lag_grid.best_estimator_
gb_gt_lag

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=120,
                          n_iter_no_change=None, presort='auto',
                          random_state=10, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
val_gb_gt_lag = gb_gt_lag_grid.cv_results_["mean_test_score"]
train_gb_gt_lag = gb_gt_lag_grid.cv_results_["mean_train_score"]
par_gb_gt_lag = gb_gt_lag_grid.cv_results_["params"]

gb_gt_lag_val_df=pd.DataFrame({'val': np.sqrt(-val_gb_gt_lag),'train':  np.sqrt(-train_gb_gt_lag), 'param': par_gb_gt_lag} )
gb_gt_lag_val_df.sort_values('val', inplace=True)
gb_gt_lag_val_df.head(20)

,val,train,param
28,702.215433,166.771788,"{'max_depth': 3, 'n_estimators': 120}"
25,702.483018,216.752333,"{'max_depth': 3, 'n_estimators': 90}"
27,702.567561,178.407291,"{'max_depth': 3, 'n_estimators': 110}"
29,702.659299,155.017753,"{'max_depth': 3, 'n_estimators': 130}"
24,702.998338,245.370712,"{'max_depth': 3, 'n_estimators': 80}"
30,703.035217,146.379700,"{'max_depth': 3, 'n_estimators': 140}"
26,703.296980,196.193058,"{'max_depth': 3, 'n_estimators': 100}"
31,703.512843,138.737724,"{'max_depth': 3, 'n_estimators': 150}"
23,708.058954,280.564712,"{'max_depth': 3, 'n_estimators': 70}"
45,713.952054,97.550925,"{'max_depth': 4, 'n_estimators': 130}"


It overfits.

In [ ]:
# make predictions
gb_gt_lag_train_yhat =gb_gt_lag.predict(Xtrain_lag)

In [ ]:
dump(gb_gt_lag, 'Diss_models/gb_gt_lag_59.joblib')

['Diss_models/gb_gt_lag_59.joblib']

**Without GT**

In [ ]:
parg_gb_mc_lag = [{'n_estimators': [1, 10, 20, 30, 40,50,60,65, 70,75, 80,90,100,110,120,130,140,150], 
              'max_depth': [2,3, 4,5, 8, 10, 15, None]}]

gb_mc_lag= GradientBoostingRegressor(random_state=10)

tscv = TimeSeriesSplit(n_splits=5)

# we'll use tscv with 5 folds 
# we also want to get the train score for later
gb_mc_lag_grid = GridSearchCV(estimator=gb_mc_lag, cv=tscv,
                           param_grid=parg_gb_mc_lag ,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# fit the estimator on the training set
gb_mc_lag_grid.fit(Xtrain_light_lag, ytrain)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=GradientBoostingRegressor(alpha=0.9,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.1,
                                                 loss='ls', max_depth=3,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_we...
                                                 presort='auto',
                                                 r

In [ ]:
# the best model's RMSE
gb_mc_lag_rmse=np.sqrt(-gb_mc_lag_grid.best_score_)
print(f'The best Gradient Boosting model has a RMSE of: {gb_mc_lag_rmse}')

The best Gradient Boosting model has a RMSE of: 712.2479479272624


In [ ]:
# the best random forest model
gb_mc_lag= gb_mc_lag_grid.best_estimator_
gb_mc_lag

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=110,
                          n_iter_no_change=None, presort='auto',
                          random_state=10, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

In [ ]:
val_gb_mc_lag = gb_mc_lag_grid.cv_results_["mean_test_score"]
train_gb_mc_lag = gb_mc_lag_grid.cv_results_["mean_train_score"]
par_gb_mc_lag = gb_mc_lag_grid.cv_results_["params"]

gb_mc_lag_val_df=pd.DataFrame({'val': np.sqrt(-val_gb_mc_lag),'train':  np.sqrt(-train_gb_mc_lag), 'param': par_gb_mc_lag} )
gb_mc_lag_val_df.sort_values('val', inplace=True)
gb_mc_lag_val_df.head(20)

,val,train,param
31,712.247948,176.726877,"{'max_depth': 3, 'n_estimators': 110}"
32,713.104163,164.663632,"{'max_depth': 3, 'n_estimators': 120}"
30,713.460179,193.633814,"{'max_depth': 3, 'n_estimators': 100}"
29,713.666906,218.108262,"{'max_depth': 3, 'n_estimators': 90}"
33,713.818423,155.793153,"{'max_depth': 3, 'n_estimators': 130}"
34,714.241646,148.839633,"{'max_depth': 3, 'n_estimators': 140}"
28,714.573816,240.988587,"{'max_depth': 3, 'n_estimators': 80}"
35,714.784238,140.455982,"{'max_depth': 3, 'n_estimators': 150}"
27,716.030395,255.576514,"{'max_depth': 3, 'n_estimators': 75}"
26,719.844376,279.621490,"{'max_depth': 3, 'n_estimators': 70}"


It overfits

In [ ]:
# make predictions
gb_mc_lag_train_yhat =gb_mc_lag.predict(Xtrain_light_lag)

In [ ]:
dump(gb_mc_lag, 'Diss_models/gb_mc_lag_59.joblib')

['Diss_models/gb_mc_lag_59.joblib']

### 2.2.3 AdaBoost

**With GT**

In [ ]:
parg_adb_gt_lag = {'n_estimators': [3, 5,10,15, 20, 50], 'learning_rate': [ 0.1, 0.25, 0.5, 0.75, 1,1.25, 1.5, 2],
     'loss' : ['linear', 'square', 'exponential']}

adb_gt_lag= AdaBoostRegressor(random_state=10)

tscv = TimeSeriesSplit(n_splits=5)

# we'll use tscv with 5 folds 
# we also want to get the train score for later
adb_gt_lag_grid = GridSearchCV(estimator=adb_gt_lag, cv=tscv,
                           param_grid=parg_adb_gt_lag ,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# fit the estimator on the training set
adb_gt_lag_grid.fit(Xtrain_lag, ytrain)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=AdaBoostRegressor(base_estimator=None, learning_rate=1.0,
                                         loss='linear', n_estimators=50,
                                         random_state=10),
             iid='warn', n_jobs=None,
             param_grid={'learning_rate': [0.1, 0.25, 0.5, 0.75, 1, 1.25, 1.5,
                                           2],
                         'loss': ['linear', 'square', 'exponential'],
                         'n_estimators': [3, 5, 10, 15, 20, 50]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='neg_mean_squared_error', verbose=0)

In [ ]:
# the best model's RMSE
adb_GT_lag_rmse=np.sqrt(-adb_gt_lag_grid.best_score_)
print(f'The best AdaBoost model has a RMSE of: {adb_GT_lag_rmse}')

The best AdaBoost model has a RMSE of: 1229.7268718906955


In [ ]:
# the best  model
adb_gt_lag= adb_gt_lag_grid.best_estimator_
adb_gt_lag

AdaBoostRegressor(base_estimator=None, learning_rate=0.75, loss='linear',
                  n_estimators=5, random_state=10)

In [ ]:
val_adb_gt_lag = adb_gt_lag_grid.cv_results_["mean_test_score"]
train_adb_gt_lag = adb_gt_lag_grid.cv_results_["mean_train_score"]
par_adb_gt_lag = adb_gt_lag_grid.cv_results_["params"]

adb_gt_lag_val_df=pd.DataFrame({'val': np.sqrt(-val_adb_gt_lag),'train':  np.sqrt(-train_adb_gt_lag), 'param': par_adb_gt_lag} )
adb_gt_lag_val_df.sort_values('val', inplace=True)
adb_gt_lag_val_df.head(20)

,val,train,param
55,1229.726872,911.003766,"{'learning_rate': 0.75, 'loss': 'linear', 'n_e..."
67,1232.023238,926.136284,"{'learning_rate': 0.75, 'loss': 'exponential',..."
51,1239.715440,876.039509,"{'learning_rate': 0.5, 'loss': 'exponential', ..."
50,1240.796717,906.204354,"{'learning_rate': 0.5, 'loss': 'exponential', ..."
22,1257.646692,857.505312,"{'learning_rate': 0.25, 'loss': 'linear', 'n_e..."
126,1259.170843,887.243772,"{'learning_rate': 2, 'loss': 'linear', 'n_esti..."
26,1259.585906,893.894629,"{'learning_rate': 0.25, 'loss': 'square', 'n_e..."
34,1259.844940,896.175160,"{'learning_rate': 0.25, 'loss': 'exponential',..."
49,1262.095499,943.899617,"{'learning_rate': 0.5, 'loss': 'exponential', ..."
17,1263.737457,889.738193,"{'learning_rate': 0.1, 'loss': 'exponential', ..."


It slightly overfits

In [ ]:
# make predictions
adb_gt_lag_train_yhat =adb_gt_lag.predict(Xtrain_lag)

In [ ]:
dump(adb_gt_lag, 'Diss_models/adb_gt_lag_59.joblib')

['Diss_models/adb_gt_lag_59.joblib']

**Without GT**

In [ ]:
parg_adb_mc_lag = {'n_estimators': [3, 5,10,15, 20, 50], 'learning_rate': [ 0.1, 0.25, 0.5, 0.75, 1,1.25, 1.5,2],
     'loss' : ['linear', 'square', 'exponential']}

adb_mc_lag= AdaBoostRegressor(random_state=10)

tscv = TimeSeriesSplit(n_splits=5)

# we'll use tscv with 5 folds 
# we also want to get the train score for later
adb_mc_lag_grid = GridSearchCV(estimator=adb_mc_lag, cv=tscv,
                           param_grid=parg_adb_mc_lag ,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# fit the estimator on the training set
adb_mc_lag_grid.fit(Xtrain_light_lag, ytrain)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=AdaBoostRegressor(base_estimator=None, learning_rate=1.0,
                                         loss='linear', n_estimators=50,
                                         random_state=10),
             iid='warn', n_jobs=None,
             param_grid={'learning_rate': [0.1, 0.25, 0.5, 0.75, 1, 1.25, 1.5,
                                           2],
                         'loss': ['linear', 'square', 'exponential'],
                         'n_estimators': [3, 5, 10, 15, 20, 50]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='neg_mean_squared_error', verbose=0)

In [ ]:
# the best model's RMSE
adb_mc_lag_rmse=np.sqrt(-adb_mc_lag_grid .best_score_)
print(f'The best AdaBoost model has a RMSE of: {adb_mc_lag_rmse}')

The best AdaBoost model has a RMSE of: 1223.1984966337448


In [ ]:
# the best  model
adb_mc_lag = adb_mc_lag_grid.best_estimator_
adb_mc_lag

AdaBoostRegressor(base_estimator=None, learning_rate=0.75, loss='exponential',
                  n_estimators=5, random_state=10)

In [ ]:
val_adb_mc_lag = adb_mc_lag_grid.cv_results_["mean_test_score"]
train_adb_mc_lag = adb_mc_lag_grid.cv_results_["mean_train_score"]
par_adb_mc_lag = adb_mc_lag_grid.cv_results_["params"]

adb_mc_lag_val_df=pd.DataFrame({'val': np.sqrt(-val_adb_mc_lag),'train':  np.sqrt(-train_adb_mc_lag), 'param': par_adb_mc_lag} )
adb_mc_lag_val_df.sort_values('val', inplace=True)
adb_mc_lag_val_df.head(20)

,val,train,param
67,1223.198497,938.041680,"{'learning_rate': 0.75, 'loss': 'exponential',..."
55,1224.802062,887.359461,"{'learning_rate': 0.75, 'loss': 'linear', 'n_e..."
73,1228.200440,896.064531,"{'learning_rate': 1, 'loss': 'linear', 'n_esti..."
50,1228.262033,905.321986,"{'learning_rate': 0.5, 'loss': 'exponential', ..."
26,1237.932459,883.150389,"{'learning_rate': 0.25, 'loss': 'square', 'n_e..."
27,1245.916774,841.658666,"{'learning_rate': 0.25, 'loss': 'square', 'n_e..."
5,1246.420823,858.276659,"{'learning_rate': 0.1, 'loss': 'linear', 'n_es..."
51,1251.801950,868.910510,"{'learning_rate': 0.5, 'loss': 'exponential', ..."
21,1252.628631,884.088539,"{'learning_rate': 0.25, 'loss': 'linear', 'n_e..."
121,1254.855865,893.785616,"{'learning_rate': 1.5, 'loss': 'exponential', ..."


Slight overfit.

In [ ]:
# make predictions
adb_mc_lag_train_yhat =adb_mc_lag.predict(Xtrain_light_lag)

In [ ]:
dump(adb_mc_lag, 'Diss_models/adb_mc_lag_59.joblib')

['Diss_models/adb_mc_lag_59.joblib']

### 2.2.4 SVR

**With GT**

In [ ]:
parg_svr_gt_lag = [{'C': [ 100,500, 1000,1500,2000,2500,3000,3500,4000,4500,5000,6000], 
                    'gamma': ["scale", "auto",0.001, 0.01, 0.1, 1, 3, 5, 10]
                   }
                  ]

svr_gt_lag = SVR(kernel="rbf")

tscv = TimeSeriesSplit(n_splits=5)

# we'll use tscv with 5 folds 
# we also want to get the train score for later
svr_gt_lag_grid = GridSearchCV(estimator=svr_gt_lag, cv=tscv,
                           param_grid=parg_svr_gt_lag ,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# fit the estimator on the training set
svr_gt_lag_grid.fit(Xtrain_lag, ytrain)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [100, 500, 1000, 1500, 2000, 2500, 3000, 3500,
                                4000, 4500, 5000, 6000],
                          'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1, 3, 5,
                                    10]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='neg_mean_squared_error', verbose=0)

In [ ]:
# the best model's RMSE
svr_GT_lag_rmse=np.sqrt(-svr_gt_lag_grid.best_score_)
print(f'The best SVR GT model has a RMSE of: {svr_GT_lag_rmse}')

The best SVR GT model has a RMSE of: 665.4269711037788


In [ ]:
# the best random forest model
svr_gt_lag= svr_gt_lag_grid.best_estimator_
svr_gt_lag

SVR(C=6000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.001,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
val_svr_gt_lag = svr_gt_lag_grid.cv_results_["mean_test_score"]
train_svr_gt_lag = svr_gt_lag_grid.cv_results_["mean_train_score"]
par_svr_gt_lag = svr_gt_lag_grid.cv_results_["params"]

svr_gt_lag_val_df=pd.DataFrame({'val': np.sqrt(-val_svr_gt_lag),'train':  np.sqrt(-train_svr_gt_lag), 'param': par_svr_gt_lag} )
svr_gt_lag_val_df.sort_values('val', inplace=True)
svr_gt_lag_val_df.head(20)

,val,train,param
101,665.426971,2432.586042,"{'C': 6000, 'gamma': 0.001}"
102,701.662268,2093.306769,"{'C': 6000, 'gamma': 0.01}"
93,732.412926,2125.807805,"{'C': 5000, 'gamma': 0.01}"
84,748.862653,2148.707347,"{'C': 4500, 'gamma': 0.01}"
75,760.768238,2181.136577,"{'C': 4000, 'gamma': 0.01}"
66,776.862085,2200.132901,"{'C': 3500, 'gamma': 0.01}"
92,778.082621,3520.067076,"{'C': 5000, 'gamma': 0.001}"
57,795.765866,2234.265974,"{'C': 3000, 'gamma': 0.01}"
48,819.422073,2333.885561,"{'C': 2500, 'gamma': 0.01}"
100,837.658738,2308.834545,"{'C': 6000, 'gamma': 'auto'}"


It underfits

In [ ]:
# make predictions
svr_gt_lag_train_yhat =svr_gt_lag.predict(Xtrain_lag)

In [ ]:
dump(svr_gt_lag, 'Diss_models/svr_gt_lag_59.joblib')

['Diss_models/svr_gt_lag_59.joblib']

**Without GT**

In [ ]:
parg_svr_mc_lag = [{'C': [1000,3000,4000,5000,6000,10000,15000,20000,30000,50000], 
                    'gamma': ["scale", "auto",0.001, 0.01, 0.1, 1, 3, 5, 10]
                   }
                  ]

svr_mc_lag = SVR(kernel="rbf")

tscv = TimeSeriesSplit(n_splits=5)

# we'll use tscv with 5 folds 
# we also want to get the train score for later
svr_mc_lag_grid = GridSearchCV(estimator=svr_mc_lag, cv=tscv,
                           param_grid=parg_svr_mc_lag ,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)

# fit the estimator on the training set
svr_mc_lag_grid.fit(Xtrain_light_lag, ytrain)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=5),
             error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [1000, 3000, 4000, 5000, 6000, 10000, 15000,
                                20000, 30000, 50000],
                          'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1, 3, 5,
                                    10]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='neg_mean_squared_error', verbose=0)

In [ ]:
# the best model's RMSE
svr_mc_lag_rmse=np.sqrt(-svr_mc_lag_grid.best_score_)
print(f'The best SVR MC model has a RMSE of: {svr_mc_lag_rmse}')

The best SVR MC model has a RMSE of: 575.9723085945724


In [ ]:
# the best random forest model
svr_mc_lag= svr_mc_lag_grid.best_estimator_
svr_mc_lag

SVR(C=50000, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma=0.001,
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
val_svr_mc_lag = svr_mc_lag_grid.cv_results_["mean_test_score"]
train_svr_mc_lag = svr_mc_lag_grid.cv_results_["mean_train_score"]
par_svr_mc_lag = svr_mc_lag_grid.cv_results_["params"]

svr_mc_lag_val_df=pd.DataFrame({'val': np.sqrt(-val_svr_mc_lag),'train':  np.sqrt(-train_svr_mc_lag), 'param': par_svr_mc_lag} )
svr_mc_lag_val_df.sort_values('val', inplace=True)
svr_mc_lag_val_df.head(20)

,val,train,param
83,575.972309,1786.530654,"{'C': 50000, 'gamma': 0.001}"
74,578.757102,1805.871363,"{'C': 30000, 'gamma': 0.001}"
65,581.135493,1818.417593,"{'C': 20000, 'gamma': 0.001}"
56,583.991226,1826.641089,"{'C': 15000, 'gamma': 0.001}"
47,586.587264,1837.907922,"{'C': 10000, 'gamma': 0.001}"
38,654.912456,2363.725530,"{'C': 6000, 'gamma': 0.001}"
39,700.076551,2084.018860,"{'C': 6000, 'gamma': 0.01}"
48,703.033217,1930.155217,"{'C': 10000, 'gamma': 0.01}"
30,715.727506,2142.713167,"{'C': 5000, 'gamma': 0.01}"
57,725.929740,1834.646700,"{'C': 15000, 'gamma': 0.01}"


It underfits.

In [ ]:
# make predictions
svr_mc_lag_train_yhat =svr_mc_lag.predict(Xtrain_light_lag)

In [ ]:
dump(svr_mc_lag, 'Diss_models/svr_mc_lag_59.joblib')

['Diss_models/svr_mc_lag_59.joblib']

### 2.2.5 RMSE

In [ ]:
rmse_persi_all

4936.789529657899

In [ ]:
RMSE2= {
    'RF With GT 1': [rf_rmse_score],
    'RF With GT 2': [rf_rmse_score2],
    'RF With GT 3': [rf_rmse_score3],
    'RF macro 1': [rf_rmse_score_lg],
    'RF macro 2': [rf_rmse_score_lg2],
    'GB with GT 1': [gb_gt_rmse1],
    'GB with GT 2': [gb_gt_rmse2],
    'GB macro 1': [gb_mc_rmse1],
    'GB macro 2': [gb_mc_rmse2],
    'ADB With GT': [adb_gt_rmse],
    'ADB macro': [adb_mc_rmse],
    'SVR with GT':[svr_gt_rmse],
    'SVR macro':[svr_mc_rmse],

'RF lag With GT': [rf_GT_lag_rmse],
    ' RF lag macro': [rf_mc_lag_rmse],
    'GB lag with GT':[ gb_GT_lag_rmse],
    'GB lag macro':[ gb_mc_lag_rmse],
'ADB lag With GT': [adb_GT_lag_rmse],
    'ADB  lag macro': [adb_mc_lag_rmse],
    'SVR  lag  with GT':[ svr_GT_lag_rmse],
    'SVR lag  macro':[ svr_mc_lag_rmse]
}

scores_df2=pd.DataFrame(RMSE2).transpose()
scores_df2.columns=['RMSE']
scores_df2["Vs Baseline"] =(scores_df2["RMSE"]/rmse_persi_all-1)*100
scores_df2


,RMSE,Vs Baseline
RF With GT 1,782.074884,-84.158229
RF With GT 2,782.074884,-84.158229
RF With GT 3,782.074884,-84.158229
RF macro 1,861.088698,-82.557719
RF macro 2,812.845615,-83.534935
GB with GT 1,721.528206,-85.384667
GB with GT 2,698.076088,-85.859715
GB macro 1,725.357502,-85.307101
GB macro 2,726.175684,-85.290528
ADB With GT,1213.456519,-75.420129


Most models overfit, except some ADB and SVR models.

***Nowcast + lagged values***

* RF: the GT model has a better RMSE than the one without GT data.
* GB: The model without GT slightly beats the one with GT data.
* ADB: The model with GT data is more accurate than the one without GT.
* SVR: the model without GT data is better than the one with.

***lagged values + all GT variables***

Each time the model with onnly macroeconomic variables is more accurate than the ones with GT data.

### 2.2.6 Feature importance

#### RF

**With GT**

In [ ]:
rf_gt_lag_fi = rf_gt_lag.feature_importances_
rf_gt_lag_df = pd.DataFrame(rf_gt_lag_fi, Xtrain_lag.columns)
rf_gt_lag_df.rename(columns={0:'Feature importance'}, inplace=True) 
rf_gt_lag_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.829676
CS_lag1,0.157184
CS_lag2,0.010680
Restaurants: 276,0.000777
DI_lag1,0.000275
cci_lag2,0.000264
Apparel: 68,0.000242
Automotive Industry: 1190,0.000215
DI_lag2,0.000159
cci_lag1,0.000087


**Without GT**

In [ ]:
rf_mc_lag_fi = rf_mc_lag.feature_importances_
rf_mc_lag_df = pd.DataFrame(rf_mc_lag_fi, Xtrain_light_lag.columns)
rf_mc_lag_df.rename(columns={0:'Feature importance'}, inplace=True) 
rf_mc_lag_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.594538
CS_lag1,0.392135
CS_lag2,0.010392
Restaurants: 276,0.000853
cci_lag1,0.000370
Automotive Industry: 1190,0.000288
Apparel: 68,0.000283
cci_lag2,0.000237
DI_lag2,0.000228
CPI_lag2,0.000181


#### GB

**With GT**

In [ ]:
gb_gt_lag_fi = gb_gt_lag.feature_importances_
gb_gt_lag_df = pd.DataFrame(gb_gt_lag_fi, Xtrain_lag.columns)
gb_gt_lag_df.rename(columns={0:'Feature importance'}, inplace=True) 
gb_gt_lag_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.806244
CS_lag1,0.181558
CS_lag2,0.009433
Restaurants: 276,0.000925
cci_lag2,0.000431
3m Bond_lag1,0.000239
cci_lag1,0.000197
Apparel: 68,0.000193
DI_lag2,0.000186
3m Bond_lag2,0.000139


**Without GT**

In [ ]:
gb_mc_lag_fi = gb_mc_lag.feature_importances_
gb_mc_lag_df = pd.DataFrame(gb_mc_lag_fi, Xtrain_light_lag.columns)
gb_mc_lag_df.rename(columns={0:'Feature importance'}, inplace=True) 
gb_mc_lag_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.806337
CS_lag1,0.181733
CS_lag2,0.009459
Restaurants: 276,0.000863
DI_lag1,0.000361
cci_lag2,0.000245
Apparel: 68,0.000231
cci_lag1,0.000224
3m Bond_lag1,0.000151
Automotive Industry: 1190,0.000092


#### AdaBoost

**With GT**

In [ ]:
adb_gt_lag_fi = adb_gt_lag.feature_importances_
adb_gt_lag_df = pd.DataFrame(adb_gt_lag_fi, Xtrain_lag.columns)
adb_gt_lag_df.rename(columns={0:'Feature importance'}, inplace=True) 
adb_gt_lag_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.842393
CS_lag1,0.122178
Restaurants: 276,0.025962
CS_lag2,0.009039
cci_lag2,0.000250
DI_lag1,0.000178
Liquor: 406,0.000000
Major Kitchen Appliances: 1293,0.000000
Mail & Package Delivery: 1150,0.000000
Lottery & Sweepstakes: 364,0.000000


**Without GT**

In [ ]:
adb_mc_lag_fi = adb_mc_lag.feature_importances_
adb_mc_lag_df = pd.DataFrame(adb_mc_lag_fi, Xtrain_light_lag.columns)
adb_mc_lag_df.rename(columns={0:'Feature importance'}, inplace=True) 
adb_mc_lag_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.597467
CS_lag1,0.389881
CS_lag2,0.007340
Restaurants: 276,0.003838
cci_lag1,0.001039
cci_lag2,0.000436
Major Kitchen Appliances: 1293,0.000000
Mail & Package Delivery: 1150,0.000000
Lottery & Sweepstakes: 364,0.000000
Kitchen & Dining: 951,0.000000


# 3. Test the models

# Nowcast + lagged values

## 3.1 Random forest

### 3.1.1 GT

In [ ]:
rf_gt_yhat_test = m_rf3.predict(Xtest)
rf_gt_mse=np.sqrt(mean_squared_error(ytest, rf_gt_yhat_test))
rf_gt_mse

1971.0002885430013

### 3.1.2 Macroeconomic variables only

In [ ]:
rf_mc_yhat_test=rf_macro2.predict(Xtest_light)
rf_mc_mse=np.sqrt(mean_squared_error(ytest_light, rf_mc_yhat_test))
rf_mc_mse

1933.6289724447236

## 3.2 Gradient Boost

### 3.2.1 With Google Trends Data

In [ ]:
gb_gt_yhat_test = gt_gb2.predict(Xtest)
gb_gt_mse=np.sqrt(mean_squared_error(ytest, gb_gt_yhat_test))
gb_gt_mse

1883.0884308861487

In [ ]:
gb_gt_mda= get_mda(ytest, gb_gt_yhat_test)
gb_gt_mda

0.0

### 3.2.2 Without Google Trends data

In [ ]:
gb_mc_yhat_test = mc_gb2.predict(Xtest_light)
gb_mc_mse=np.sqrt(mean_squared_error(ytest_light, gb_mc_yhat_test))
gb_mc_mse

1848.77716375944

In [ ]:
gb_mc_mda= get_mda(ytest_light, gb_mc_yhat_test)
gb_mc_mda

0.0

## 3.3 Adaboost

### 3.3.1 WIth Google Trends data

In [ ]:
adb_gt_yhat_test = adb_gt_1.predict(Xtest)
adb_gt_mse=np.sqrt(mean_squared_error(ytest, adb_gt_yhat_test ))
adb_gt_mse

2522.1916646917434

### 3.3.2 Macroeconomic variables

In [ ]:
adb_mc_yhat_test = adb_mc.predict(Xtest_light)
adb_mc_mse=np.sqrt(mean_squared_error(ytest_light, adb_mc_yhat_test ))
adb_mc_mse

2311.2443100423225

## 3.4 Support Vector Regressor

### 3.4.1 With Google Trends

In [ ]:
svr_gt_yhat_test = svr_gt_1.predict(Xtest)
svr_gt_mse=np.sqrt(mean_squared_error(ytest, svr_gt_yhat_test ))
svr_gt_mse

3906.092565294355

### 3.4.2 Macroeconomic variables

In [ ]:
svr_mc_yhat_test = svr_mc_1.predict(Xtest_light)
svr_mc_mse=np.sqrt(mean_squared_error(ytest_light, svr_mc_yhat_test ))
svr_mc_mse

3811.560694645522

# Lagged values

## RF

***With GT***

In [ ]:
lag_rf_gt_yh_test = rf_gt_lag.predict(Xtest_lag)
lag_rf_gt_mse=np.sqrt(mean_squared_error(ytest, lag_rf_gt_yh_test))
lag_rf_gt_mse

2027.7262205962197

***With MC***

In [ ]:
lag_rf_mc_yh_test=rf_mc_lag.predict(Xtest_light_lag)
lag_rf_mc_mse=np.sqrt(mean_squared_error(ytest_light, lag_rf_mc_yh_test))
lag_rf_mc_mse

2551.903511917924

## GB

***With GT***

In [ ]:
lag_gb_gt_yh_test = gb_gt_lag.predict(Xtest_lag)
lag_gb_gt_mse=np.sqrt(mean_squared_error(ytest, lag_gb_gt_yh_test))
lag_gb_gt_mse

1949.560531517774

***With MC***

In [ ]:
lag_gb_mc_yh_test=gb_mc_lag.predict(Xtest_light_lag)
lag_gb_mc_mse=np.sqrt(mean_squared_error(ytest_light, lag_gb_mc_yh_test))
lag_gb_mc_mse

1935.090912877314

## ADB

***With GT***

In [ ]:
lag_adb_gt_yh_test = adb_gt_lag.predict(Xtest_lag)
lag_adb_gt_mse=np.sqrt(mean_squared_error(ytest, lag_adb_gt_yh_test))
lag_adb_gt_mse

2543.203280399069

***With MC***

In [ ]:
lag_adb_mc_yh_test=adb_mc_lag.predict(Xtest_light_lag)
lag_adb_mc_mse=np.sqrt(mean_squared_error(ytest_light, lag_adb_mc_yh_test))
lag_adb_mc_mse


2744.9656738842796

## SVR

***With GT***

In [ ]:
lag_svr_gt_yh_test = svr_gt_lag.predict(Xtest_lag)
lag_svr_gt_mse=np.sqrt(mean_squared_error(ytest, lag_svr_gt_yh_test))
lag_svr_gt_mse

4052.748221654096

***With MC***

In [ ]:
lag_svr_mc_yh_test=svr_mc_lag.predict(Xtest_light_lag)
lag_svr_mc_mse=np.sqrt(mean_squared_error(ytest_light, lag_svr_mc_yh_test))
lag_svr_mc_mse

3808.882102742771

## 3.5 RMSE

In [ ]:
RMSE_test= {
    'RF With GT': [rf_gt_mse], 
    'RF macro': [rf_mc_mse],
    
    'GB with GT': [gb_gt_mse],
    'GB macro': [gb_mc_mse],
    
    'ADB With GT': [adb_gt_mse],
    'ADB macro': [adb_mc_mse],
   
    'SVR With GT': [svr_gt_mse],
    'SVR macro ': [svr_mc_mse],
    
    'Lagged RF With GT': [lag_rf_gt_mse], 
    'Lagged RF macro': [lag_rf_mc_mse],
    
    'Lagged GB with GT': [lag_gb_gt_mse],
    'Lagged GB macro': [lag_gb_mc_mse],
    
    'Lagged ADB With GT': [lag_adb_gt_mse],
    'Lagged ADB macro': [lag_adb_mc_mse],
   
    'Lagged SVR With GT': [lag_svr_gt_mse],
    'Lagged SVR macro ': [lag_svr_mc_mse],
    
}

scores_df_test=pd.DataFrame(RMSE_test).transpose()
scores_df_test.columns=['RMSE Test']
scores_df_test["Vs Baseline"] =(scores_df_test["RMSE Test"]/rmse_persi_all-1)*100
scores_df_test

,RMSE Test,Vs Baseline
RF With GT,1971.000289,-60.075262
RF macro,1933.628972,-60.832258
GB with GT,1883.088431,-61.856011
GB macro,1848.777164,-62.551023
ADB With GT,2522.191665,-48.910286
ADB macro,2311.244310,-53.183252
SVR With GT,3906.092565,-20.877879
SVR macro,3811.560695,-22.792724
Lagged RF With GT,2027.726221,-58.926217
Lagged RF macro,2551.903512,-48.308440


On the test set, we have interesting results:
* RF, the model with GT got a higher/worse RMSE than the one without GT
* GB, the model without GT did better than the one with GT.
* ADB, the model without GT did better than the one with GT.
* SVR, the model without GT did better than the ones with it.
The models with the lowest RMSE were the GB ones.

For models with only lagged variables, except GT:
* RF: model with GT's RMSE > (better) Macro
* GB: model with GT's RMSE < Macro
* ADB: model with GT's RMSE > Macro
* SVR: model with GT's RMSE < Macro
Once again, the models with the lowest RMSE were the GB ones.

We removed the current actual values of macroeconomic variables in the second type of models and it only slightly increased the RMSE.

## 3.6 **Analysis of the true and predicted values for each category**

We want to have a look at the RMSE of each category for each model with and without the Google trends data.

We need first to get the name of the categories and append it to the ytest dataset.

### 3.6.1 Preparation of the ytest dataframe with the names of the categories

In [ ]:
ytest.head()

,CS
quarter,
2017Q1,5769
2017Q2,5878
2017Q3,5549
2017Q4,5965
2018Q1,6703


In [ ]:
ytest.shape

(430, 1)

In [ ]:
Col_Xtest=Xtest.copy()
Col_Xtest.head()

,GT,GT_lag1,GT_lag2,DI,DI_lag1,DI_lag2,3m Bond,3m Bond_lag1,3m Bond_lag2,cci,...,Restaurants: 276,Small Kitchen Appliances: 1292,Sporting Goods: 263,Tobacco Products: 123,Transportation & Logistics: 50,Vehicle Fuels & Lubricants: 1269,Vehicle Maintenance: 138,Vehicle Parts & Accessories: 89,Water Supply & Treatment: 1349,Wine: 405
quarter,,,,,,,,,,,,,,,,,,,,,
2017Q1,0.628668,0.000000,0.000000,1.911960,0.000000,0.000000,0.202656,0.000000,0.000000,0.624696,...,0,0,0,0,0,0,0,0,0,0
2017Q2,0.344156,0.628668,0.000000,1.593379,1.911960,0.000000,0.129086,0.202656,0.000000,0.539322,...,0,0,0,0,0,0,0,0,0,0
2017Q3,0.581249,0.344156,0.628668,1.847164,1.593379,1.911960,0.235197,0.129086,0.202656,0.434545,...,0,0,0,0,0,0,0,0,0,0
2017Q4,-1.125820,0.581249,0.344156,1.998355,1.847164,1.593379,0.559189,0.235197,0.129086,0.453949,...,0,0,0,0,0,0,0,0,0,0
2018Q1,0.723505,-1.125820,0.581249,2.041553,1.998355,1.847164,0.210438,0.559189,0.235197,0.554845,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
Col_Xtest_pred=Col_Xtest.loc[:,['GT','GT_lag1','GT_lag2',
                                            'DI','DI_lag1','DI_lag2',
                                            'cci','cci_lag1','cci_lag2',
                                            'vix','vix_lag1','vix_lag2',
                                            'CS_lag1','CS_lag2']]
Col_Xtest_pred.head()

,GT,GT_lag1,GT_lag2,DI,DI_lag1,DI_lag2,cci,cci_lag1,cci_lag2,vix,vix_lag1,vix_lag2,CS_lag1,CS_lag2
quarter,,,,,,,,,,,,,,
2017Q1,0.628668,0.000000,0.000000,1.911960,0.000000,0.000000,0.624696,0.000000,0.000000,-0.528409,0.000000,0.000000,0.000000,0.000000
2017Q2,0.344156,0.628668,0.000000,1.593379,1.911960,0.000000,0.539322,0.624696,0.000000,-0.023662,-0.528409,0.000000,0.172919,0.000000
2017Q3,0.581249,0.344156,0.628668,1.847164,1.593379,1.911960,0.434545,0.539322,0.624696,-0.219770,-0.023662,-0.528409,0.184929,0.172919
2017Q4,-1.125820,0.581249,0.344156,1.998355,1.847164,1.593379,0.453949,0.434545,0.539322,-0.200027,-0.219770,-0.023662,0.148678,0.184929
2018Q1,0.723505,-1.125820,0.581249,2.041553,1.998355,1.847164,0.554845,0.453949,0.434545,0.644288,-0.200027,-0.219770,0.194515,0.148678


In [ ]:
Col_Xtest.drop(['GT','GT_lag1','GT_lag2',
                     'DI','DI_lag1','DI_lag2',
                     'cci','cci_lag1','cci_lag2',
                      'vix','vix_lag1','vix_lag2',
                      'CS_lag1','CS_lag2'],axis=1, inplace=True)
Col_Xtest.head()

,3m Bond,3m Bond_lag1,3m Bond_lag2,CPI,CPI_lag1,CPI_lag2,Air Travel: 203,Animal Products & Services: 882,Apartments & Residential Rentals: 378,Apparel: 68,...,Restaurants: 276,Small Kitchen Appliances: 1292,Sporting Goods: 263,Tobacco Products: 123,Transportation & Logistics: 50,Vehicle Fuels & Lubricants: 1269,Vehicle Maintenance: 138,Vehicle Parts & Accessories: 89,Water Supply & Treatment: 1349,Wine: 405
quarter,,,,,,,,,,,,,,,,,,,,,
2017Q1,0.202656,0.000000,0.000000,-0.013212,0.000000,0.000000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017Q2,0.129086,0.202656,0.000000,1.404905,-0.013212,0.000000,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017Q3,0.235197,0.129086,0.202656,-0.013212,1.404905,-0.013212,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017Q4,0.559189,0.235197,0.129086,0.797140,-0.013212,1.404905,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018Q1,0.210438,0.559189,0.235197,-0.823565,0.797140,-0.013212,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
Col_Xtest.drop(['3m Bond', '3m Bond_lag1','3m Bond_lag2'],axis=1, inplace=True)
Col_Xtest.head()

,CPI,CPI_lag1,CPI_lag2,Air Travel: 203,Animal Products & Services: 882,Apartments & Residential Rentals: 378,Apparel: 68,Audio Equipment: 361,Auto Insurance: 467,Automotive Industry: 1190,...,Restaurants: 276,Small Kitchen Appliances: 1292,Sporting Goods: 263,Tobacco Products: 123,Transportation & Logistics: 50,Vehicle Fuels & Lubricants: 1269,Vehicle Maintenance: 138,Vehicle Parts & Accessories: 89,Water Supply & Treatment: 1349,Wine: 405
quarter,,,,,,,,,,,,,,,,,,,,,
2017Q1,-0.013212,0.000000,0.000000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017Q2,1.404905,-0.013212,0.000000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017Q3,-0.013212,1.404905,-0.013212,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017Q4,0.797140,-0.013212,1.404905,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018Q1,-0.823565,0.797140,-0.013212,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
Col_Xtest.drop(['CPI', 'CPI_lag1','CPI_lag2'],axis=1, inplace=True)
Col_Xtest.head()

,Air Travel: 203,Animal Products & Services: 882,Apartments & Residential Rentals: 378,Apparel: 68,Audio Equipment: 361,Auto Insurance: 467,Automotive Industry: 1190,Baked Goods: 907,Beer: 404,Bicycles & Accessories: 1191,...,Restaurants: 276,Small Kitchen Appliances: 1292,Sporting Goods: 263,Tobacco Products: 123,Transportation & Logistics: 50,Vehicle Fuels & Lubricants: 1269,Vehicle Maintenance: 138,Vehicle Parts & Accessories: 89,Water Supply & Treatment: 1349,Wine: 405
quarter,,,,,,,,,,,,,,,,,,,,,
2017Q1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017Q2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017Q3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2017Q4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2018Q1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
dummies_cat=Col_Xtest.idxmax(axis=1)
dummies_cat.head()

quarter
2017Q1    Air Travel: 203
2017Q2    Air Travel: 203
2017Q3    Air Travel: 203
2017Q4    Air Travel: 203
2018Q1    Air Travel: 203
dtype: object

In [ ]:
dummies_df=pd.DataFrame({'date':dummies_cat.index, 'Cat':dummies_cat.values})
dummies_df.set_index('date', inplace=True)
dummies_df.head()

,Cat
date,
2017Q1,Air Travel: 203
2017Q2,Air Travel: 203
2017Q3,Air Travel: 203
2017Q4,Air Travel: 203
2018Q1,Air Travel: 203


We merge the ytest dataframe with the dataframe containing the names of the categories.

In [ ]:
ytest_cat=pd.concat([ytest,dummies_df], axis=1)
ytest_cat.head()

,CS,Cat
2017Q1,5769,Air Travel: 203
2017Q2,5878,Air Travel: 203
2017Q3,5549,Air Travel: 203
2017Q4,5965,Air Travel: 203
2018Q1,6703,Air Travel: 203


We save the file.

In [ ]:
ytest_cat.to_csv('ytest_cat.csv')

### 3.6.2 Random Forests

In [ ]:
rf_gt_yhat_test_df=pd.DataFrame(rf_gt_yhat_test)
rf_gt_yhat_test_df.columns = ['pred']
rf_gt_yhat_test_df.head()

,pred
0,2259.28237
1,5877.48000
2,5963.20000
3,5584.71000
4,6160.80000


In [ ]:
rf_gt_ytest=ytest.copy()
rf_gt_ytest.head()

,CS
quarter,
2017Q1,5769
2017Q2,5878
2017Q3,5549
2017Q4,5965
2018Q1,6703


In [ ]:
print(rf_gt_ytest.shape)
print(rf_gt_yhat_test_df.shape)

(430, 1)
(430, 1)


In [ ]:
rf_gt_ytest=ytest_cat.copy()
rf_gt_ytest.reset_index(inplace=True)
rf_gt_ytest.head()

,index,CS,Cat
0,2017Q1,5769,Air Travel: 203
1,2017Q2,5878,Air Travel: 203
2,2017Q3,5549,Air Travel: 203
3,2017Q4,5965,Air Travel: 203
4,2018Q1,6703,Air Travel: 203


We merge the ytest dataframe with the dataframe with the predicted results from the Random Forests models with and without the GT data.

In [ ]:
# Merging the ytest with the predicted results
rf_gt_best=pd.merge(rf_gt_ytest,rf_gt_yhat_test_df['pred'], how = 'left',left_index = True, right_index = True)
rf_gt_best.head()

,index,CS,Cat,pred
0,2017Q1,5769,Air Travel: 203,2259.28237
1,2017Q2,5878,Air Travel: 203,5877.48000
2,2017Q3,5549,Air Travel: 203,5963.20000
3,2017Q4,5965,Air Travel: 203,5584.71000
4,2018Q1,6703,Air Travel: 203,6160.80000


In [ ]:
rf_mc_yhat_test_df=pd.DataFrame(rf_mc_yhat_test)
rf_mc_yhat_test_df.columns = ['pred_no GT']
rf_mc_yhat_test_df.head()

,pred_no GT
0,2259.282370
1,5838.880000
2,6016.546667
3,5629.543333
4,6225.266667


In [ ]:
rf_RMSE=pd.merge(rf_gt_best,rf_mc_yhat_test_df['pred_no GT'], how = 'left',left_index = True, right_index = True)
rf_RMSE.head()

,index,CS,Cat,pred,pred_no GT
0,2017Q1,5769,Air Travel: 203,2259.28237,2259.282370
1,2017Q2,5878,Air Travel: 203,5877.48000,5838.880000
2,2017Q3,5549,Air Travel: 203,5963.20000,6016.546667
3,2017Q4,5965,Air Travel: 203,5584.71000,5629.543333
4,2018Q1,6703,Air Travel: 203,6160.80000,6225.266667


In [ ]:
lag_rf_gt_yh_test_df=pd.DataFrame(lag_rf_gt_yh_test)
lag_rf_gt_yh_test_df.columns = ['Lag pred_GT']
#Transform the predicted results for the model with the MC values into a dataframe
lag_rf_mc_yh_test_df=pd.DataFrame(lag_rf_mc_yh_test)
lag_rf_mc_yh_test_df.columns = ['Lag pred_MC']

# Merge the lagged dataframes
lag_rf_RMSE=pd.merge(lag_rf_gt_yh_test_df,lag_rf_mc_yh_test_df['Lag pred_MC'], how = 'left',left_index = True, right_index = True)

# Merge it with the initial datafram
rf_RMSE_FV=pd.merge(rf_RMSE,lag_rf_RMSE, how = 'left',left_index = True, right_index = True)
rf_RMSE_FV.head()

,index,CS,Cat,pred,pred_no GT,Lag pred_GT,Lag pred_MC
0,2017Q1,5769,Air Travel: 203,2259.28237,2259.282370,2007.282612,1670.360216
1,2017Q2,5878,Air Travel: 203,5877.48000,5838.880000,5945.800000,5847.660606
2,2017Q3,5549,Air Travel: 203,5963.20000,6016.546667,6043.866667,6020.379847
3,2017Q4,5965,Air Travel: 203,5584.71000,5629.543333,5589.995590,5713.883929
4,2018Q1,6703,Air Travel: 203,6160.80000,6225.266667,6130.248485,6030.465963


In [ ]:
rmse_rf_gt_cat=[]
rmse_rf_mc_cat=[]
rmse_rf_lag_gt_cat=[]
rmse_rf_lag_mc_cat=[]

def rmse_cat( g ):
    rmse_gt = np.sqrt(mean_squared_error(g['CS'][1:], g['pred'][1:]))
    rmse_rf_gt_cat.append(rmse_gt)
    rmse_mc=np.sqrt(mean_squared_error(g['CS'][1:], g['pred_no GT'][1:]))
    rmse_rf_mc_cat.append(rmse_mc)
    
    rmse_lag_gt = np.sqrt(mean_squared_error(g['CS'][1:], g['Lag pred_GT'][1:]))
    rmse_rf_lag_gt_cat.append( rmse_lag_gt )
    rmse_lag_mc=np.sqrt(mean_squared_error(g['CS'][1:], g['Lag pred_MC'][1:]))
    rmse_rf_lag_mc_cat.append(rmse_lag_mc)
    return pd.Series(dict({'RMSE GT':rmse_gt, 'RMSE no GT' : rmse_mc,
                           'Lag RMSE GT':rmse_lag_gt, 'Lag RMSE no GT' : rmse_lag_mc }))

In [ ]:
rf_RMSE_scores=rf_RMSE_FV.groupby('Cat').apply(rmse_cat).reset_index()
rf_RMSE_scores

,Cat,RMSE GT,RMSE no GT,Lag RMSE GT,Lag RMSE no GT
0,Air Travel: 203,515.074919,471.840462,422.851092,402.103159
1,Animal Products & Services: 882,44.610849,42.584748,38.731247,50.247661
2,Apartments & Residential Rentals: 378,7757.611813,6499.728482,7812.715066,8693.791929
3,Apparel: 68,451.390819,359.832751,592.846531,598.689768
4,Audio Equipment: 361,73.636791,77.356356,61.773480,48.602753
5,Auto Insurance: 467,45.745618,35.338717,28.081391,28.684656
6,Automotive Industry: 1190,528.007565,397.284899,474.462763,510.456935
7,Baked Goods: 907,841.860960,841.667043,895.709021,770.400075
8,Beer: 404,67.427695,74.975928,64.487702,55.869225
9,Bicycles & Accessories: 1191,60.694160,65.492527,58.592112,58.950669


In [ ]:
rf_RMSE_scores['No Gt vs GT']=rf_RMSE_scores['RMSE GT']-rf_RMSE_scores['RMSE no GT']
rf_RMSE_scores['% No Gt vs GT']=(rf_RMSE_scores['RMSE GT']/rf_RMSE_scores['RMSE no GT']-1)*100

rf_RMSE_scores['Lag No Gt vs GT']=rf_RMSE_scores['Lag RMSE GT']-rf_RMSE_scores['Lag RMSE no GT']
rf_RMSE_scores['% Lag No Gt vs GT']=(rf_RMSE_scores['Lag RMSE GT']/rf_RMSE_scores['Lag RMSE no GT']-1)*100

rf_RMSE_scores.head()

,Cat,RMSE GT,RMSE no GT,Lag RMSE GT,Lag RMSE no GT,No Gt vs GT,% No Gt vs GT,Lag No Gt vs GT,% Lag No Gt vs GT
0,Air Travel: 203,515.074919,471.840462,422.851092,402.103159,43.234457,9.162940,20.747933,5.159853
1,Animal Products & Services: 882,44.610849,42.584748,38.731247,50.247661,2.026101,4.757810,-11.516414,-22.919305
2,Apartments & Residential Rentals: 378,7757.611813,6499.728482,7812.715066,8693.791929,1257.883331,19.352860,-881.076863,-10.134552
3,Apparel: 68,451.390819,359.832751,592.846531,598.689768,91.558069,25.444618,-5.843238,-0.976004
4,Audio Equipment: 361,73.636791,77.356356,61.773480,48.602753,-3.719565,-4.808350,13.170727,27.098726


In [ ]:
#creation of a style to highlight the categories 
def color_rmse(val):
    """
    Takes a scalar and returns a string with
    the css property `'color: green'` if RMSE of GT< to RMSE of macroeconomic variables.
    Retunr red otherwise
    """
    color = 'green' if val > 0 else 'red'
    return 'color: %s' % color

In [ ]:
rf_RMSE_scores.sort_values(by=['% No Gt vs GT'], ascending=True)

,Cat,RMSE GT,RMSE no GT,Lag RMSE GT,Lag RMSE no GT,No Gt vs GT,% No Gt vs GT,Lag No Gt vs GT,% Lag No Gt vs GT
30,Mobile & Wireless: 382,8.747544,26.616868,20.470937,21.917633,-17.869324,-67.135336,-1.446696,-6.600604
32,Non-Alcoholic Beverages: 560,63.105634,93.488474,78.555531,107.066302,-30.382840,-32.499023,-28.510772,-26.629080
37,Transportation & Logistics: 50,82.508603,113.878337,70.266209,104.117293,-31.369734,-27.546709,-33.851084,-32.512451
19,Hobbies & Leisure: 65,140.245608,173.935576,138.740761,183.301285,-33.689968,-19.369222,-44.560524,-24.309990
14,Drugs & Medications: 646,71.640831,88.004510,74.051393,97.337779,-16.363679,-18.594137,-23.286386,-23.923276
29,Meat & Seafood: 909,109.444871,131.969365,119.755391,89.743815,-22.524495,-17.067972,30.011577,33.441388
17,Fruits & Vegetables: 908,251.281677,299.718688,199.312290,121.483364,-48.437011,-16.160824,77.828926,64.065501
26,Lottery & Sweepstakes: 364,143.934180,167.405033,118.954312,224.800053,-23.470853,-14.020398,-105.845742,-47.084393
12,Clothing Accessories: 124,88.644898,101.890841,94.694393,171.978098,-13.245943,-13.000131,-77.283706,-44.938109
15,Education: 74,316.226481,352.095362,201.222178,106.625792,-35.868881,-10.187263,94.596386,88.718108


**Random Forests**

***With GT***
* Mostly durable and services in the TOP 10. The GT model worked really well for "afforadable" means of transportations: motorcycles, transportation and logistics, bicycles. 
* In the bottom 10, mostly services. It didn't work well for important investments such as apartment rentals, restaurants, automative infustry and air travel.

***Without GT***
* The model with the macroeconomic variables performed better than the GT one for expensive purchases such as apartment rentals and the purchase of car or camera and photo equipment.
* It is more precise for non-durables.
* The model without GT data performed better with the services and durables.

It is interesting to see that both models did not manage to predict correctly the same categories.

GT model is more precise for 16 categories out of 43, mostly are non durables.

In [ ]:
rf_RMSE_scores.to_csv('rf_RMSE_scores.csv')

### 3.6.2 Gradient Boosting

In [ ]:
#Transform the predicted results for the model with the Gt values into a dataframe
gb_gt_yhat_test_df=pd.DataFrame(gb_gt_yhat_test)
gb_gt_yhat_test_df.columns = ['pred']
gb_gt_yhat_test_df.head()

,pred
0,3183.762709
1,5621.261906
2,6101.834108
3,5598.835998
4,6080.128625


In [ ]:
#Do the same for the predicted results without the GT values
gb_mc_yhat_test_df=pd.DataFrame(gb_mc_yhat_test)
gb_mc_yhat_test_df.columns = ['pred_no GT']
gb_mc_yhat_test_df.head()

,pred_no GT
0,2928.610918
1,5846.976316
2,5993.334504
3,5770.723270
4,6144.380758


In [ ]:
gb_gt_ytest=ytest_cat.copy()
gb_gt_ytest.reset_index(inplace=True)
gb_gt_ytest.head()

,index,CS,Cat
0,2017Q1,5769,Air Travel: 203
1,2017Q2,5878,Air Travel: 203
2,2017Q3,5549,Air Travel: 203
3,2017Q4,5965,Air Travel: 203
4,2018Q1,6703,Air Travel: 203


In [ ]:
# Merging the ytest with the predicted results
gb_gt_best=pd.merge(gb_gt_ytest,gb_gt_yhat_test_df['pred'], how = 'left',left_index = True, right_index = True)
gb_RMSE=pd.merge(gb_gt_best,gb_mc_yhat_test_df['pred_no GT'], how = 'left',left_index = True, right_index = True)

lag_gb_gt_yh_test_df=pd.DataFrame(lag_gb_gt_yh_test)
lag_gb_gt_yh_test_df.columns = ['Lag pred_GT']
#Transform the predicted results for the model with the MC values into a dataframe
lag_gb_mc_yh_test_df=pd.DataFrame(lag_gb_mc_yh_test)
lag_gb_mc_yh_test_df.columns = ['Lag pred_MC']

# Merge the lagged dataframes
lag_gb_RMSE=pd.merge(lag_gb_gt_yh_test_df,lag_gb_mc_yh_test_df['Lag pred_MC'], how = 'left',left_index = True, right_index = True)

# Merge it with the initial datafram
gb_RMSE_FV=pd.merge(gb_RMSE,lag_gb_RMSE, how = 'left',left_index = True, right_index = True)
gb_RMSE_FV.head()

gb_RMSE_FV.head()

,index,CS,Cat,pred,pred_no GT,Lag pred_GT,Lag pred_MC
0,2017Q1,5769,Air Travel: 203,3183.762709,2928.610918,2177.324147,2102.596890
1,2017Q2,5878,Air Travel: 203,5621.261906,5846.976316,5506.797275,5409.374907
2,2017Q3,5549,Air Travel: 203,6101.834108,5993.334504,5971.748537,5944.832800
3,2017Q4,5965,Air Travel: 203,5598.835998,5770.723270,5616.931937,5613.574222
4,2018Q1,6703,Air Travel: 203,6080.128625,6144.380758,6041.821392,6046.142303


In [ ]:
gb_RMSE_scores=gb_RMSE_FV.groupby('Cat').apply(rmse_cat).reset_index()

gb_RMSE_scores['No Gt vs GT']=gb_RMSE_scores['RMSE GT']-gb_RMSE_scores['RMSE no GT']
gb_RMSE_scores['% No Gt vs GT']=(gb_RMSE_scores['RMSE GT']/gb_RMSE_scores['RMSE no GT']-1)*100

gb_RMSE_scores['Lag No Gt vs GT']=gb_RMSE_scores['Lag RMSE GT']-gb_RMSE_scores['Lag RMSE no GT']
gb_RMSE_scores['% Lag No Gt vs GT']=(gb_RMSE_scores['Lag RMSE GT']/gb_RMSE_scores['Lag RMSE no GT']-1)*100

gb_RMSE_scores

,Cat,RMSE GT,RMSE no GT,Lag RMSE GT,Lag RMSE no GT,No Gt vs GT,% No Gt vs GT,Lag No Gt vs GT,% Lag No Gt vs GT
0,Air Travel: 203,424.920848,322.866802,408.949450,415.698294,102.054046,31.608715,-6.748844,-1.623496
1,Animal Products & Services: 882,81.505794,57.200679,88.649008,92.372610,24.305115,42.490954,-3.723602,-4.031067
2,Apartments & Residential Rentals: 378,6529.001063,7673.294109,7714.867136,7235.505343,-1144.293046,-14.912670,479.361793,6.625132
3,Apparel: 68,324.552592,310.289237,389.849485,325.583556,14.263355,4.596793,64.265929,19.738690
4,Audio Equipment: 361,42.994241,33.469126,56.683990,64.143204,9.525115,28.459408,-7.459215,-11.629002
5,Auto Insurance: 467,46.269493,42.519308,51.021123,52.755465,3.750185,8.819957,-1.734341,-3.287510
6,Automotive Industry: 1190,462.389622,429.624352,531.230465,423.054337,32.765270,7.626493,108.176129,25.570268
7,Baked Goods: 907,386.380122,448.784454,418.714778,499.668555,-62.404332,-13.905190,-80.953778,-16.201495
8,Beer: 404,108.511613,68.749345,72.718582,75.542086,39.762268,57.836577,-2.823504,-3.737657
9,Bicycles & Accessories: 1191,56.823076,40.070528,108.894942,72.246081,16.752549,41.807656,36.648861,50.727818


In [ ]:
gb_RMSE_scores.sort_values(by=['% No Gt vs GT'], ascending=True, inplace=True)
gb_RMSE_scores

,Cat,RMSE GT,RMSE no GT,Lag RMSE GT,Lag RMSE no GT,No Gt vs GT,% No Gt vs GT,Lag No Gt vs GT,% Lag No Gt vs GT
35,Sporting Goods: 263,65.861807,118.728493,118.508692,66.977427,-52.866686,-44.527379,51.531265,76.938258
15,Education: 74,153.486033,245.607411,177.257413,220.490002,-92.121379,-37.507573,-43.232589,-19.607505
21,Home Furnishings: 270,363.201447,435.689906,427.085854,500.537784,-72.488459,-16.637626,-73.451930,-14.674602
2,Apartments & Residential Rentals: 378,6529.001063,7673.294109,7714.867136,7235.505343,-1144.293046,-14.912670,479.361793,6.625132
7,Baked Goods: 907,386.380122,448.784454,418.714778,499.668555,-62.404332,-13.905190,-80.953778,-16.201495
25,Liquor: 406,53.439863,60.756851,59.624144,68.301628,-7.316988,-12.043067,-8.677484,-12.704651
33,Restaurants: 276,1809.505812,2038.209825,2031.627602,2117.182164,-228.704013,-11.220828,-85.554562,-4.040964
10,Camera & Photo Equipment: 573,79.733225,85.253049,77.212586,89.016669,-5.519824,-6.474636,-11.804083,-13.260531
32,Non-Alcoholic Beverages: 560,77.272048,76.500791,101.362272,101.444884,0.771257,1.008169,-0.082612,-0.081436
3,Apparel: 68,324.552592,310.289237,389.849485,325.583556,14.263355,4.596793,64.265929,19.738690


**Gradient Boosting**

***Model with GT data***
* In the top five, there are mostly services and in the top 10, mostly durables and services. Part of them are linked with transportations or vehicle parts. We also find utilities, home financing which are paid recurrently.
* In the bottom 5/10, we find again apartment rentals which are expensive and restaurants and the purchase which can be expensive. We find again air travel and surprisingly electricity (vs top 5), apprel, hotels and accomodations and vehicle maintenance.
* It is more precise than the one with only macroeconomic variables for 24 categories out of 43.

***Model without GT***
* In the top 5, we find mainly services and durables, and in the top 10 we find mainly services and a bit of durables. 4 categories are linked with vehicles and transportations and are more affordable than the purchase of a car. We also find insurances, home financing and utilities, those payments are generally recurring and the same amount.
* The bottom 5/10 is similar to the one with the GT data.

The biggest gaps between GT and Macroeconomic variables in absolute values are mostly services, especially apartments and residential rentals.

In [ ]:
gb_RMSE_scores.to_csv('gb_RMSE_scores.csv')

### 3.6.3 Adaboost

In [ ]:
#Transform the predicted results for the model with the Gt values into a dataframe
adb_gt_yhat_test_df=pd.DataFrame(adb_gt_yhat_test)
adb_gt_yhat_test_df.columns = ['pred']
adb_gt_yhat_test_df.head()

,pred
0,4196.942134
1,4196.942134
2,4196.942134
3,4196.942134
4,4196.942134


In [ ]:
#Do the same for the predicted results without the GT values
adb_mc_yhat_test_df=pd.DataFrame(adb_mc_yhat_test)
adb_mc_yhat_test_df.columns = ['pred_no GT']
adb_mc_yhat_test_df.head()

,pred_no GT
0,4198.358524
1,4198.358524
2,4198.358524
3,4198.358524
4,4198.358524


In [ ]:
adb_gt_ytest=ytest_cat.copy()
adb_gt_ytest.reset_index(inplace=True)
adb_gt_ytest.head()

,index,CS,Cat
0,2017Q1,5769,Air Travel: 203
1,2017Q2,5878,Air Travel: 203
2,2017Q3,5549,Air Travel: 203
3,2017Q4,5965,Air Travel: 203
4,2018Q1,6703,Air Travel: 203


In [ ]:
# Merging the ytest with the predicted results
adb_gt_best=pd.merge(adb_gt_ytest,adb_gt_yhat_test_df['pred'], how = 'left',left_index = True, right_index = True)
adb_RMSE=pd.merge(adb_gt_best,adb_mc_yhat_test_df['pred_no GT'], how = 'left',left_index = True, right_index = True)

lag_adb_gt_yh_test_df=pd.DataFrame(lag_adb_gt_yh_test)
lag_adb_gt_yh_test_df.columns = ['Lag pred_GT']
#Transform the predicted results for the model with the MC values into a dataframe
lag_adb_mc_yh_test_df=pd.DataFrame(lag_adb_mc_yh_test)
lag_adb_mc_yh_test_df.columns = ['Lag pred_MC']

# Merge the lagged dataframes
lag_adb_RMSE=pd.merge(lag_adb_gt_yh_test_df,lag_adb_mc_yh_test_df['Lag pred_MC'], how = 'left',left_index = True, right_index = True)

# Merge it with the initial datafram
adb_RMSE_FV=pd.merge(adb_RMSE,lag_adb_RMSE, how = 'left',left_index = True, right_index = True)
adb_RMSE_FV.head()

,index,CS,Cat,pred,pred_no GT,Lag pred_GT,Lag pred_MC
0,2017Q1,5769,Air Travel: 203,4196.942134,4198.358524,3837.636364,3837.636364
1,2017Q2,5878,Air Travel: 203,4196.942134,4198.358524,3837.636364,3837.636364
2,2017Q3,5549,Air Travel: 203,4196.942134,4198.358524,3837.636364,3837.636364
3,2017Q4,5965,Air Travel: 203,4196.942134,4198.358524,3837.636364,3837.636364
4,2018Q1,6703,Air Travel: 203,4196.942134,4198.358524,4087.427598,3837.636364


In [ ]:
adb_RMSE_scores=adb_RMSE_FV.groupby('Cat').apply(rmse_cat).reset_index()
adb_RMSE_scores['No Gt vs GT']=adb_RMSE_scores['RMSE GT']-adb_RMSE_scores['RMSE no GT']
adb_RMSE_scores['% No Gt vs GT']=(adb_RMSE_scores['RMSE GT']/adb_RMSE_scores['RMSE no GT']-1)*100

adb_RMSE_scores['Lag No Gt vs GT']=adb_RMSE_scores['Lag RMSE GT']-adb_RMSE_scores['Lag RMSE no GT']
adb_RMSE_scores['% Lag No Gt vs GT']=(adb_RMSE_scores['Lag RMSE GT']/adb_RMSE_scores['Lag RMSE no GT']-1)*100


adb_RMSE_scores.sort_values(by=['% No Gt vs GT'], ascending=True, inplace=True)
adb_RMSE_scores

,Cat,RMSE GT,RMSE no GT,Lag RMSE GT,Lag RMSE no GT,No Gt vs GT,% No Gt vs GT,Lag No Gt vs GT,% Lag No Gt vs GT
41,Water Supply & Treatment: 1349,19.297281,90.946954,140.611513,201.999672,-71.649673,-78.781829,-61.388158,-30.390227
40,Vehicle Parts & Accessories: 89,59.846468,100.631629,158.671306,217.186660,-40.785162,-40.529168,-58.515354,-26.942426
4,Audio Equipment: 361,62.914960,101.442946,160.929567,219.186751,-38.527986,-37.979956,-58.257184,-26.578789
7,Baked Goods: 907,63.883739,97.134044,341.723766,341.723766,-33.250305,-34.231360,0.000000,0.000000
22,Home Insurance: 465,175.489106,253.006253,95.575744,110.420553,-77.517147,-30.638431,-14.844809,-13.443882
26,Lottery & Sweepstakes: 364,159.755392,227.598253,403.692904,403.692904,-67.842862,-29.808164,0.000000,0.000000
35,Sporting Goods: 263,264.550552,347.622367,140.419133,101.254863,-83.071815,-23.897143,39.164269,38.678902
21,Home Furnishings: 270,215.171274,281.344178,381.861404,381.861404,-66.172904,-23.520268,0.000000,0.000000
10,Camera & Photo Equipment: 573,282.032200,368.237809,144.342184,87.612535,-86.205609,-23.410309,56.729649,64.750608
20,Home Financing: 466,339.587155,426.449216,199.023616,137.874277,-86.862061,-20.368676,61.149339,44.351521


**Adaboost**

***Model with GT data***
* In the top 5, we find mostly non durables and a few durable and in the top 10, mostly non durables, a few durables and services.
* In the bottom 5, we find mostly services, a few non durables and durables, again apartments and rentals, restaurants, vehicle mainenance, air travel, which can be really expensive.

***Model without GT***
* The top and bottom 5 and 10 are the same than the model with GT.

62% of the categories have a better RMSE with the model without GT data.
The model with GT data is more precise with non durables and the one without it is more accurate with durables. 

In [ ]:
adb_RMSE_scores.to_csv('adb_RMSE_scores.csv')

### 3.6.4 SVR

In [ ]:
#Transform the predicted results for the model with the Gt values into a dataframe
svr_gt_yhat_test_df=pd.DataFrame(svr_gt_yhat_test)
svr_gt_yhat_test_df.columns = ['pred']
svr_gt_yhat_test_df.head()

,pred
0,4160.023240
1,5084.586858
2,5867.691036
3,5718.619925
4,5783.572231


In [ ]:
#Do the same for the predicted results without the GT values
svr_mc_yhat_test_df=pd.DataFrame(svr_mc_yhat_test)
svr_mc_yhat_test_df.columns = ['pred_no GT']
svr_mc_yhat_test_df.head()

,pred_no GT
0,4141.858694
1,5181.958000
2,5851.630612
3,5670.592149
4,5794.158837


In [ ]:
svr_gt_ytest=ytest_cat.copy()
svr_gt_ytest.reset_index(inplace=True)
svr_gt_ytest.head()

,index,CS,Cat
0,2017Q1,5769,Air Travel: 203
1,2017Q2,5878,Air Travel: 203
2,2017Q3,5549,Air Travel: 203
3,2017Q4,5965,Air Travel: 203
4,2018Q1,6703,Air Travel: 203


In [ ]:
# Merging the ytest with the predicted results
svr_gt_best=pd.merge(svr_gt_ytest,svr_gt_yhat_test_df['pred'], how = 'left',left_index = True, right_index = True)
svr_RMSE=pd.merge(svr_gt_best,svr_mc_yhat_test_df['pred_no GT'], how = 'left',left_index = True, right_index = True)

lag_svr_gt_yh_test_df=pd.DataFrame(lag_svr_gt_yh_test)
lag_svr_gt_yh_test_df.columns = ['Lag pred_GT']
#Transform the predicted results for the model with the MC values into a dataframe
lag_svr_mc_yh_test_df=pd.DataFrame(lag_svr_mc_yh_test)
lag_svr_mc_yh_test_df.columns = ['Lag pred_MC']

# Merge the lagged dataframes
lag_svr_RMSE=pd.merge(lag_svr_gt_yh_test_df,lag_svr_mc_yh_test_df['Lag pred_MC'], how = 'left',left_index = True, right_index = True)

# Merge it with the initial datafram
svr_RMSE_FV=pd.merge(svr_RMSE,lag_svr_RMSE, how = 'left',left_index = True, right_index = True)
svr_RMSE_FV.head()

,index,CS,Cat,pred,pred_no GT,Lag pred_GT,Lag pred_MC
0,2017Q1,5769,Air Travel: 203,4160.023240,4141.858694,4175.426738,4206.070785
1,2017Q2,5878,Air Travel: 203,5084.586858,5181.958000,4970.939179,5185.107659
2,2017Q3,5549,Air Travel: 203,5867.691036,5851.630612,5788.565379,5866.792120
3,2017Q4,5965,Air Travel: 203,5718.619925,5670.592149,5662.174508,5677.717966
4,2018Q1,6703,Air Travel: 203,5783.572231,5794.158837,5723.249601,5811.908343


In [ ]:
svr_RMSE_scores=svr_RMSE_FV.groupby('Cat').apply(rmse_cat).reset_index()
svr_RMSE_scores['No Gt vs GT']=svr_RMSE_scores['RMSE GT']-svr_RMSE_scores['RMSE no GT']
svr_RMSE_scores['% No Gt vs GT']=(svr_RMSE_scores['RMSE GT']/svr_RMSE_scores['RMSE no GT']-1)*100

svr_RMSE_scores['Lag No Gt vs GT']=svr_RMSE_scores['Lag RMSE GT']-svr_RMSE_scores['Lag RMSE no GT']
svr_RMSE_scores['% Lag No Gt vs GT']=(svr_RMSE_scores['Lag RMSE GT']/svr_RMSE_scores['Lag RMSE no GT']-1)*100


svr_RMSE_scores.sort_values(by=['% No Gt vs GT'], ascending=True, inplace=True)
svr_RMSE_scores

,Cat,RMSE GT,RMSE no GT,Lag RMSE GT,Lag RMSE no GT,No Gt vs GT,% No Gt vs GT,Lag No Gt vs GT,% Lag No Gt vs GT
12,Clothing Accessories: 124,140.267550,157.306567,153.154752,142.896313,-17.039017,-10.831726,10.258440,7.178939
7,Baked Goods: 907,42.773130,46.923572,55.353470,36.592608,-4.150442,-8.845111,18.760862,51.269540
21,Home Furnishings: 270,128.737310,133.796654,149.649735,126.366984,-5.059344,-3.781368,23.282750,18.424710
16,Electricity: 658,274.186819,271.013946,282.995815,270.783810,3.172873,1.170742,12.212005,4.509873
0,Air Travel: 203,438.109912,414.296031,475.620478,409.684330,23.813882,5.748035,65.936148,16.094379
26,Lottery & Sweepstakes: 364,122.948335,115.165257,149.631596,109.708042,7.783078,6.758182,39.923554,36.390727
36,Tobacco Products: 123,152.239039,138.714372,165.719822,123.049580,13.524667,9.750011,42.670242,34.677275
23,Hotels & Accommodations: 179,308.341839,280.353668,331.441397,280.417520,27.988170,9.983165,51.023877,18.195681
17,Fruits & Vegetables: 908,330.971400,294.621543,375.800671,291.872016,36.349858,12.337814,83.928655,28.755294
29,Meat & Seafood: 909,243.143398,215.409984,263.439694,207.307921,27.733414,12.874711,56.131773,27.076521


**SVR**

***With Gt***
* Same as the ADB model, the top and bottom categories are the same.
* In the top 10, we find mostly mostly services and durables.
* In the bottom 10, we find mostly services.
* More precise than the one without GT for 31 categories out of 43, mostly services and non durables.

***Without Gt***
* Model more precise for non durables.


In [ ]:
svr_RMSE_scores.to_csv('svr_RMSE_scores.csv')

## Feature importance

### Random Forests

#### With GT

In [ ]:
rf_gt_fin = m_rf3.feature_importances_
rf_gt_df = pd.DataFrame(rf_gt_fin, Xtest.columns)
rf_gt_df.rename(columns={0:'Feature importance'}, inplace=True) 
rf_gt_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.829594
CS_lag1,0.154538
CS_lag2,0.012466
Restaurants: 276,0.000777
cci_lag1,0.000418
cci,0.000324
cci_lag2,0.000307
DI,0.000304
Apparel: 68,0.000242
DI_lag1,0.000229


#### Without GT

In [ ]:
rf_mc_fin = rf_macro2.feature_importances_
rf_mc_df = pd.DataFrame(rf_mc_fin, Xtest_light.columns)
rf_mc_df.rename(columns={0:'Feature importance'}, inplace=True) 
rf_mc_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.829595
CS_lag1,0.160717
CS_lag2,0.006274
Restaurants: 276,0.000777
cci,0.000490
cci_lag1,0.000368
DI,0.000342
Apparel: 68,0.000242
cci_lag2,0.000238
DI_lag1,0.000226


### Gradient Boosting

# Load the models
from joblib import load

rf_gt_best=load('Diss_models/rf_gt_n2.joblib')
rf_mc_best = load('Diss_models/rf_mc_n2.joblib')

gb_gt_best=load('Diss_models/gt_gb_n1.joblib')
gb_mc_best=load('Diss_models/gt_mc_n1.joblib')

adb_gt_best=load('Diss_models/adb_gt_3m.joblib')
adb_mc_best=load('Diss_models/adb_mc_3m.joblib')

svr_gt_best=load('Diss_models/svr_gt_1.joblib')
svr_mc_best=load('Diss_models/svr_mc_1.joblib')

#### With GT

In [ ]:
gb_gt_fin = gt_gb2.feature_importances_
gb_gt_df = pd.DataFrame(gb_gt_fin, Xtest.columns)
gb_gt_df.rename(columns={0:'Feature importance'}, inplace=True) 
gb_gt_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.806302
CS_lag1,0.186479
CS_lag2,0.004093
Restaurants: 276,0.000964
DI_lag2,0.000371
cci,0.000363
cci_lag2,0.000307
Apparel: 68,0.000264
3m Bond,0.000163
cci_lag1,0.000136


#### Without GT

In [ ]:
gb_mc_fin = mc_gb2.feature_importances_
gb_mc_df = pd.DataFrame(gb_mc_fin, Xtest_light.columns)
gb_mc_df.rename(columns={0:'Feature importance'}, inplace=True) 
gb_mc_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.834073
CS_lag1,0.152731
CS_lag2,0.009581
Restaurants: 276,0.000973
cci_lag1,0.000446
DI_lag1,0.000386
Apparel: 68,0.000378
DI,0.000309
Automotive Industry: 1190,0.000235
cci,0.000160


### Adaboost

#### With GT

In [ ]:
adb_gt_fin = adb_gt_1.feature_importances_
adb_gt_df = pd.DataFrame(adb_gt_fin, Xtest.columns)
adb_gt_df.rename(columns={0:'Feature importance'}, inplace=True) 
adb_gt_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.851604
CS_lag1,0.134580
CS_lag2,0.010075
Restaurants: 276,0.002378
cci_lag1,0.000601
DI,0.000269
cci_lag2,0.000268
DI_lag1,0.000225
Health Insurance: 249,0.000000
Hobbies & Leisure: 65,0.000000


#### Without GT

In [ ]:
adb_mc_fin = adb_mc.feature_importances_
adb_mc_df = pd.DataFrame(adb_mc_fin, Xtest_light.columns)
adb_mc_df.rename(columns={0:'Feature importance'}, inplace=True) 
adb_mc_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.745111
CS_lag1,0.211348
Restaurants: 276,0.029203
CS_lag2,0.006899
Apparel: 68,0.006301
DI,0.000373
cci_lag1,0.000231
cci_lag2,0.000157
cci,0.000140
DI_lag2,0.000086


### With lagged values

### RF

#### With GT

In [ ]:
lag_rf_gt_fin = rf_gt_lag.feature_importances_
lag_rf_gt_df = pd.DataFrame(lag_rf_gt_fin, Xtest_lag.columns)
lag_rf_gt_df.rename(columns={0:'Feature importance'}, inplace=True) 
lag_rf_gt_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.829676
CS_lag1,0.157184
CS_lag2,0.010680
Restaurants: 276,0.000777
DI_lag1,0.000275
cci_lag2,0.000264
Apparel: 68,0.000242
Automotive Industry: 1190,0.000215
DI_lag2,0.000159
cci_lag1,0.000087


#### Without GT

In [ ]:
lag_rf_mc_fin = rf_mc_lag.feature_importances_
lag_rf_mc_df = pd.DataFrame(lag_rf_mc_fin, Xtest_light_lag.columns)
lag_rf_mc_df.rename(columns={0:'Feature importance'}, inplace=True) 
lag_rf_mc_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.594538
CS_lag1,0.392135
CS_lag2,0.010392
Restaurants: 276,0.000853
cci_lag1,0.000370
Automotive Industry: 1190,0.000288
Apparel: 68,0.000283
cci_lag2,0.000237
DI_lag2,0.000228
CPI_lag2,0.000181


### GB

#### With GT

In [ ]:
lag_gb_gt_fin = gb_gt_lag.feature_importances_
lag_gb_gt_df = pd.DataFrame(lag_gb_gt_fin, Xtest_lag.columns)
lag_gb_gt_df.rename(columns={0:'Feature importance'}, inplace=True) 
lag_gb_gt_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.806244
CS_lag1,0.181558
CS_lag2,0.009433
Restaurants: 276,0.000925
cci_lag2,0.000431
3m Bond_lag1,0.000239
cci_lag1,0.000197
Apparel: 68,0.000193
DI_lag2,0.000186
3m Bond_lag2,0.000139


#### Without GT

In [ ]:
lag_gb_mc_fin = gb_mc_lag.feature_importances_
lag_gb_mc_df = pd.DataFrame(lag_gb_mc_fin, Xtest_light_lag.columns)
lag_gb_mc_df.rename(columns={0:'Feature importance'}, inplace=True) 
lag_gb_mc_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.806337
CS_lag1,0.181733
CS_lag2,0.009459
Restaurants: 276,0.000863
DI_lag1,0.000361
cci_lag2,0.000245
Apparel: 68,0.000231
cci_lag1,0.000224
3m Bond_lag1,0.000151
Automotive Industry: 1190,0.000092


### AdaBoost

#### With GT

In [ ]:
lag_adb_gt_fin = adb_gt_lag.feature_importances_
lag_adb_gt_df = pd.DataFrame(lag_adb_gt_fin, Xtest_lag.columns)
lag_adb_gt_df.rename(columns={0:'Feature importance'}, inplace=True) 
lag_adb_gt_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.842393
CS_lag1,0.122178
Restaurants: 276,0.025962
CS_lag2,0.009039
cci_lag2,0.000250
DI_lag1,0.000178
Liquor: 406,0.000000
Major Kitchen Appliances: 1293,0.000000
Mail & Package Delivery: 1150,0.000000
Lottery & Sweepstakes: 364,0.000000


#### Without GT

In [ ]:
lag_adb_mc_fin = adb_mc_lag.feature_importances_
lag_adb_mc_df = pd.DataFrame(lag_adb_mc_fin, Xtest_light_lag.columns)
lag_adb_mc_df.rename(columns={0:'Feature importance'}, inplace=True) 
lag_adb_mc_df.sort_values('Feature importance', ascending=False).nlargest(20,'Feature importance')

,Feature importance
Apartments & Residential Rentals: 378,0.597467
CS_lag1,0.389881
CS_lag2,0.007340
Restaurants: 276,0.003838
cci_lag1,0.001039
cci_lag2,0.000436
Major Kitchen Appliances: 1293,0.000000
Mail & Package Delivery: 1150,0.000000
Lottery & Sweepstakes: 364,0.000000
Kitchen & Dining: 951,0.000000
